<a href="https://colab.research.google.com/github/MrinalA2009/ZEDD/blob/main/Zero_Shot_Embedding_Drift_Detection_A_Lightweight_Defense_Against_Prompt_Injection_in_Instruction_Following_LLMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%autosave 30

Autosaving every 30 seconds


# Install Necessary Modules

In [ ]:
!pip install datasets==2.15.0 openai fasttext tqdm numpy==1.25.0
!pip install transformers==4.44.0 sentence-transformers==3.0.1
!rm -rf ~/.cache/huggingface/datasets
!rm -rf /root/.cache/huggingface/datasets

  Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
treescope 0.1.10 requires numpy>=1.25.2, but you have numpy 1.25.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.25.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.0 which is incompatible.
tensorflow 2.19.0 requires nump

  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.0
    Uninstalling numpy-1.25.0:
      Successfully uninstalled numpy-1.25.0
ERROR: Operation cancelled by user
^C


#Load and Filter Data

In [ ]:
def calculate_avg_length(dataset):
    phase1 = dataset["Phase1"]
    phase2 = dataset["Phase2"]

    phase1_total = sum(len(body.strip()) for body in phase1["body"])
    phase2_total = sum(len(body.strip()) for body in phase2["body"])

    total_rows = len(phase1["body"]) + len(phase2["body"])
    return (phase1_total + phase2_total) / total_rows

In [ ]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_KEY"))

In [ ]:
from datasets import load_dataset, Dataset

try:
    dataset_injected = load_dataset("microsoft/llmail-inject-challenge")
except NotImplementedError:
    print("Loading from cache failed, attempting to force download.")
    dataset_injected = load_dataset("microsoft/llmail-inject-challenge", download_mode="force_redownload")

In [ ]:
print(calculate_avg_length(dataset_injected))

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd

def remove_duplicates_from_dataset_dict(dataset_dict):

    cleaned_dict = {}
    seen = set()

    for phase_name, dataset in dataset_dict.items():
        print(f"\n=== Processing {phase_name} ===")
        print(f"Original size: {len(dataset)}")


        df = dataset.to_pandas()
        df['body_normalized'] = df['body'].str.replace(r'\s+', '', regex=True).str.lower()

        mask = ~df['body_normalized'].isin(seen)
        df_filtered = df[mask].copy()


        df_deduped = df_filtered.drop_duplicates(subset=['body_normalized'], keep='first')


        seen.update(df_deduped['body_normalized'].tolist())


        df_deduped = df_deduped.drop('body_normalized', axis=1)

        print(f"After deduplication: {len(df_deduped)} (removed {len(df) - len(df_deduped)} duplicates)")


        cleaned_dict[phase_name] = Dataset.from_pandas(df_deduped)

    return DatasetDict(cleaned_dict)

dataset_injected = remove_duplicates_from_dataset_dict(dataset_injected)

In [ ]:
print(calculate_avg_length(dataset_injected))

In [ ]:
!wget -q https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz

import fasttext
from datasets import DatasetDict
from collections import defaultdict

ft_model = fasttext.load_model("lid.176.ftz")

def fasttext_detect_language(text, threshold=0.05):
    try:
        if not text or not isinstance(text, str):
            return 'unknown'

        clean_text = text.strip()
        if not clean_text:
            return 'unknown'


        clean_text = ' '.join(clean_text.split())

        prediction = ft_model.predict(clean_text, k=1)
        label, prob = prediction[0][0], prediction[1][0]

        detected_lang = label.replace('__label__', '')

        return detected_lang
    except Exception as e:
        return 'unknown'


def make_filter_fn():
    local_counter = defaultdict(int)
    def _filter(entry):
        body = entry.get("body", "")
        if not isinstance(body, str) or len(body.strip()) == 0:
            local_counter["removed"] += 1
            return False
        if 'system' in body.lower() or '<<' in body.lower():
            local_counter["kept_system"] += 1
            return True
        lang = fasttext_detect_language(body)
        if lang == 'en':
            local_counter["kept_en"] += 1
            return True
        else:
            local_counter["removed"] += 1
            return False
    return _filter, local_counter

def filter_english(dataset_dict):
    filtered_dict = {}
    for phase, dataset in dataset_dict.items():
        print(f"\nFiltering {phase}...")
        filter_fn, counter = make_filter_fn()
        filtered_dataset = dataset.filter(
            filter_fn,
            desc=f"Filtering {phase}",
            num_proc=1,
            with_indices=False
        )
        total = len(dataset)
        kept = counter["kept_en"] + counter["kept_system"]
        removed = counter["removed"]
        print(f"{phase} Summary:")
        print(f"  Total entries:        {total:,}")
        print(f"  Kept (English):       {counter['kept_en']:,}")
        print(f"  Kept (system):        {counter['kept_system']:,}")
        print(f"  Removed (non-English): {removed:,} ({(removed/total)*100:.2f}%)")
        filtered_dict[phase] = filtered_dataset
    return DatasetDict(filtered_dict)

In [ ]:
dataset_fasttext = filter_english(dataset_injected)

In [ ]:
dataset_injected = dataset_fasttext.remove_columns('__index_level_0__')

In [ ]:
dataset_injected

In [ ]:
print(calculate_avg_length(dataset_injected))

In [ ]:
def split_dataset_dict(dataset_dict):

    dataset_injected_first = {}
    dataset_injected_second = {}
    dataset_injected_third = {}
    dataset_injected_fourth = {}

    for phase_name, dataset in dataset_dict.items():
        # Calculate split points (divide into 4 equal parts)
        total_rows = len(dataset)
        quarter = total_rows // 4

        # Calculate split indices
        split_1 = quarter
        split_2 = quarter * 2
        split_3 = quarter * 3

        # Split the dataset into 4 parts
        first_quarter = dataset.select(range(0, split_1))
        second_quarter = dataset.select(range(split_1, split_2))
        third_quarter = dataset.select(range(split_2, split_3))
        fourth_quarter = dataset.select(range(split_3, total_rows))

        # Add to respective dictionaries
        dataset_injected_first[phase_name] = first_quarter
        dataset_injected_second[phase_name] = second_quarter
        dataset_injected_third[phase_name] = third_quarter
        dataset_injected_fourth[phase_name] = fourth_quarter

        print(f"{phase_name}: Split {total_rows} rows into {len(first_quarter)} + {len(second_quarter)} + {len(third_quarter)} + {len(fourth_quarter)} rows")

    # Convert dictionaries back to DatasetDict objects
    dataset_injected_first = DatasetDict(dataset_injected_first)
    dataset_injected_second = DatasetDict(dataset_injected_second)
    dataset_injected_third = DatasetDict(dataset_injected_third)
    dataset_injected_fourth = DatasetDict(dataset_injected_fourth)

    return dataset_injected_first, dataset_injected_second, dataset_injected_third, dataset_injected_fourth

In [ ]:
dataset_injected_first, dataset_injected_second, dataset_injected_third, dataset_injected_fourth = split_dataset_dict(dataset_injected)

In [ ]:
def split_dataset(dataset_dict, number=16):
    result = []
    for phase_name, dataset in dataset_dict.items():
        res = {}
        total_rows = len(dataset)
        quarter = total_rows // number

        for i in range(number):
          if i==(number-1):
              split_val = dataset.select(range(quarter*i, len(dataset)))
          else:
              split_val = dataset.select(range(quarter*i, quarter*(i+1)))
          res[phase_name] = split_val

          result.append(DatasetDict(res))

    return result

# Create JSONL File for OpenAI Batch API

In [ ]:
from datasets import Dataset, DatasetDict
import time
import json
counter = 0
def create_jsonl_file(batch, field, output_file="batch_requests.jsonl", model="gpt-3.5-turbo-0125"):
    global counter
    text = batch[field]
    if not text:
        return "Error with Texts"

    try:
        counter += 1
        custom_id = f"batch-{counter}"

        request_data = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": model,
                "messages": [
                    {
                        "role": "system",
                        "content": """You are a cybersecurity expert specializing in prompt injection classification.

                                      You will receive injected prompts and must classify them into one of these attack categories:

                                      1. **jailbreak** — Only prompts designed to bypass the model's safety restrictions, override ethical constraints, or make the AI ignore its built-in instructions. Often involves pretending to be someone else, using reverse psychology, or roleplay.
                                      2. **system leak** — Only prompts aimed at exposing hidden system messages, configurations, internal logic, or developer instructions that are not meant for the user.
                                      3. **task override** — Only prompts that attempt to change or override the original objective assigned to the model. The attacker tries to reprogram the task, shift the topic, or redefine the goal.
                                      4. **encoding manipulation** — Only prompts that hide malicious instructions using THESE ENCODED FORMATS (base64, hex, Unicode, URL encoding) to bypass filters or detection systems.
                                      5. **prompt confusion** — Only prompts that use misleading, contradictory, repetitive wording, or ambiguous instructions to confuse the model's behavior. These attacks often involve multiple, conflicting tasks or embedded triggers.

                                      Respond with only one category name (e.g., "jailbreak", "system_leak", "task_override", "encoding_manipulation", "prompt_confusion") with no markdown or any symbols before the category name."""
                    },
                    {
                        "role": "user",
                        "content": f"You will receive a prompt. For that prompt, respond with the category (one of: jailbreak, system_leak, task_override, encoding_manipulation, prompt_confusion) in that format. Take into account the exact definitions for each type of injection and do not return that the prompt isn't injected. Here is the prompt:\n\n {text}"
                    }
                ],
                "max_tokens": 20,
                "temperature": 0.1
            }
        }

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(request_data) + '\n')


    except Exception as e:
        print(f"Error creating batch file: {e}")

def allocate_dataset(dataset_injected, field, batch_size=1, output_file="batch_requests.jsonl"):

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("")

    for split_name, dataset in dataset_injected.items():
        print(f"Processing {split_name} split with {len(dataset)} samples...")

        dataset.map (
            lambda batch: create_jsonl_file(batch, field, output_file),
            batched=True,
            batch_size=batch_size,
            desc=f"Creating batch requests for {split_name}"
        )

    print(f"All batch requests written to {output_file}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Use the OpenAI Batch API to allocate categories

In [ ]:
allocate_dataset(dataset_injected_first, "body")

In [ ]:
batch_input_file = client.files.create(
    file=open("batch_requests.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch_input_file_id = batch_input_file.id
batch_val = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "first quarter"
    }
)

In [ ]:
batch = client.batches.retrieve(batch_val.id)
print(batch)
batch_output_file_id = batch.output_file_id


In [ ]:
file_response = client.files.content("file-91tJR5n1NNFxLWSyuLnPsc")

In [ ]:
print(file_response.text)

In [ ]:
error_file_response = client.files.content(batch.error_file_id)
print(error_file_response.text)

In [ ]:
def split_dataset_dict_half(dataset_dict):

    dataset_injected_first = {}
    dataset_injected_second = {}

    for phase_name, dataset in dataset_dict.items():
        # Calculate split point (divide into 2 equal parts)
        total_rows = len(dataset)
        half = total_rows // 2

        # Split the dataset into 2 parts
        first_half = dataset.select(range(0, half))
        second_half = dataset.select(range(half, total_rows))

        # Add to respective dictionaries
        dataset_injected_first[phase_name] = first_half
        dataset_injected_second[phase_name] = second_half

        print(f"{phase_name}: Split {total_rows} rows into {len(first_half)} + {len(second_half)} rows")

    # Convert dictionaries back to DatasetDict objects
    dataset_injected_first = DatasetDict(dataset_injected_first)
    dataset_injected_second = DatasetDict(dataset_injected_second)

    return dataset_injected_first, dataset_injected_second

In [ ]:
dataset_injected_second1, dataset_injected_second2 = split_dataset_dict_half(dataset_injected_second)

In [ ]:
allocate_dataset(dataset_injected_second1, "body", output_file="batch_requests_second1.jsonl")

In [ ]:
batch_input_file_second1 = client.files.create(
    file=open("batch_requests_second1.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_second1)

In [ ]:
batch_input_file_id_second1 = batch_input_file_second1.id
batch_val_second1 = client.batches.create(
    input_file_id=batch_input_file_id_second1,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "second part 1"
    }
)

In [ ]:
batch_second1 = client.batches.retrieve(batch_val_second1.id)
print(batch_second1)
batch_output_file_id_second1 = batch_second1.output_file_id

In [ ]:
file_response_second1 = client.files.content("file-PnZn49p5zhdykW6SjD3P18")


In [ ]:
file_response_second1.text

In [ ]:
allocate_dataset(dataset_injected_second2, "body", output_file="batch_requests_second2.jsonl")

In [ ]:
batch_input_file_second2 = client.files.create(
    file=open("batch_requests_second2.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_second2)

In [ ]:
batch_input_file_id_second2 = batch_input_file_second2.id
batch_val_second2 = client.batches.create(
    input_file_id=batch_input_file_id_second2,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "second part 2"
    }
)

In [ ]:
batch_second2 = client.batches.retrieve(batch_val_second2.id)
print(batch_second2)
batch_output_file_id_second2 = batch_second2.output_file_id

In [ ]:
file_response_second2 = client.files.content("file-Fm4x3QvzpEjRnTxkNdvebQ")

In [ ]:
file_response_second2.text

In [ ]:
allocate_dataset(dataset_injected_third, "body", output_file="batch_requests_third.jsonl")

In [ ]:
batch_input_file_third = client.files.create(
    file=open("batch_requests_third.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_third)

In [ ]:
batch_input_file_id_third = batch_input_file_third.id
batch_val_third = client.batches.create(
    input_file_id=batch_input_file_id_third,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "third part"
    }
)

In [ ]:
# batch_third = client.batches.retrieve(batch_val_third.id)
batch_third = client.batches.retrieve("batch_6899767108408190a2f675b9a2730a93")
print(batch_third)
batch_output_file_id_third = batch_third.output_file_id

In [ ]:
# List all batches to find your batch
batches = client.batches.list()
for batch in batches.data:
    print(f"Batch ID: {batch.id}, Status: {batch.status}, Description: {batch.metadata.get('description', 'N/A')}")

In [ ]:
# Check if the variable exists and what it contains
print(f"batch_output_file_id_third = {batch_output_file_id_third}")

In [ ]:
file_response_third = client.files.content(batch_output_file_id_third)

In [ ]:
allocate_dataset(dataset_injected_fourth, "body", output_file="batch_requests_fourth.jsonl")

In [ ]:
batch_input_file_fourth = client.files.create(
    file=open("batch_requests_fourth.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_fourth)

In [ ]:
batch_input_file_id_fourth = batch_input_file_fourth.id
batch_val_fourth = client.batches.create(
    input_file_id=batch_input_file_id_fourth,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "fourth part"
    }
)

In [ ]:
batch_fourth = client.batches.retrieve(batch_val_fourth.id)
print(batch_fourth)
batch_output_file_id_fourth = batch_fourth.output_file_id

In [ ]:
file_response_fourth = client.files.content("file-XQCuwbotifK7REVpRKkd47")

In [ ]:
print(file_response_fourth.text)

# Process Batch

In [ ]:
def process_batch_and_add_categories(original_dataset, batch_content, batch_size=1, filter_failed=True):
    """
    Process batch responses and add categories to dataset

    Args:
        original_dataset: HuggingFace DatasetDict
        batch_content: String containing batch responses (JSON/JSONL format)
        batch_size: Size of each batch (default=1)
        filter_failed: Whether to filter out failed entries (default=True)
    """
    from datasets import DatasetDict, Dataset

    print("Starting batch processing...")
    print(f"Batch size: {batch_size}")
    print(f"Filter failed: {filter_failed}")


    # Step 1: Parse the batch content
    print("Parsing batch responses...")
    batch_responses = {}

    # Parse the batch content
    batch_data = parse_batch_content(batch_content)

    if not batch_data:
        print("ERROR: No batch data could be parsed from the input!")
        print(f"Input content preview: {repr(batch_content[:500] if batch_content else 'None')}...")
        return original_dataset

    print(f"Successfully parsed {len(batch_data)} batch responses")

    # Process each response
    for response_data in batch_data:
        try:
            custom_id = response_data['custom_id']

            # Extract content from response
            content = response_data['response']['body']['choices'][0]['message']['content']

            # Parse categories from content
            categories = []

            # For batch_size=1, we expect a single category
            if batch_size == 1:
                category = extract_category_from_text(content)
                if category and is_valid_category(category):
                    categories = [category]
                else:
                    print(f"Invalid category extracted from {custom_id}: '{category}' from content: '{content}'")
                    categories = ['failed_parsing']
            else:
                # Handle multiple categories (your existing logic)
                content_lines = content.split('\n')

                for line_content in content_lines:
                    line_content = line_content.strip()
                    if not line_content:
                        continue

                    # Handle multiple categories in one line
                    if ',' in line_content or '=' in line_content:
                        potential_parts = []
                        for sep in [',', '=', ';', '|']:
                            if sep in line_content:
                                potential_parts = line_content.split(sep)
                                break

                        if potential_parts:
                            for part in potential_parts:
                                part = part.strip()
                                if part:
                                    category = extract_category_from_text(part)
                                    if category and is_valid_category(category):
                                        categories.append(category)
                        continue

                    # Single category per line
                    category = extract_category_from_text(line_content)
                    if category and is_valid_category(category):
                        categories.append(category)

                # Validate we have the expected number of categories
                if len(categories) != batch_size:
                    print(f"Expected {batch_size} categories for {custom_id}, but found {len(categories)}")
                    print(f"Categories found: {categories}")
                    print(f"Raw content: {repr(content)}")

                    # Adjust categories list
                    if len(categories) < batch_size:
                        missing = batch_size - len(categories)
                        categories.extend(['failed_parsing'] * missing)
                    else:
                        categories = categories[:batch_size]

            batch_responses[custom_id] = categories
            print(f"Batch {custom_id}: Found {len(categories)} categories: {categories}")

        except Exception as e:
            print(f"Error parsing batch response: {e}")
            print(f"Response data keys: {list(response_data.keys()) if isinstance(response_data, dict) else 'Not a dict'}")
            if isinstance(response_data, dict):
                print(f"Custom ID: {response_data.get('custom_id', 'MISSING')}")
            continue

    print(f"Found {len(batch_responses)} successful batches")

    # Step 2: Process each split
    updated_splits = {}
    failed_batches = set()

    # First, let's see what custom_ids we actually have
    available_custom_ids = sorted(batch_responses.keys())
    print(f"Available custom_ids: {available_custom_ids[:10]}..." if len(available_custom_ids) > 10 else f"Available custom_ids: {available_custom_ids}")

    # Extract just the batch numbers to understand the sequence
    batch_numbers = []
    for custom_id in available_custom_ids:
        try:
            parts = custom_id.split('-')
            if len(parts) >= 2:
                batch_num = int(parts[1])
                batch_numbers.append(batch_num)
        except ValueError:
            continue

    if batch_numbers:
        print(f"Batch number range: {min(batch_numbers)} to {max(batch_numbers)} ({len(batch_numbers)} total)")

    for split_name, dataset in original_dataset.items():
        print(f"\nProcessing {split_name}...")

        split_categories = []
        num_samples = len(dataset)

        print(f"Dataset has {num_samples} samples")

        # Create a mapping based on available custom_ids
        # We'll match them in order to the dataset samples
        sorted_custom_ids = sorted(batch_responses.keys(), key=lambda x: int(x.split('-')[1]) if len(x.split('-')) > 1 and x.split('-')[1].isdigit() else 0)

        samples_processed = 0
        custom_id_index = 0

        # Go through samples and match with available batch responses
        for i in range(0, num_samples, batch_size):
            current_batch_size = min(batch_size, num_samples - i)

            if custom_id_index < len(sorted_custom_ids):
                # Use the next available custom_id
                custom_id = sorted_custom_ids[custom_id_index]
                custom_id_index += 1

                if custom_id in batch_responses:
                    batch_cats = batch_responses[custom_id]

                    # Ensure we have the right number of categories
                    if len(batch_cats) == current_batch_size:
                        split_categories.extend(batch_cats)
                        print(f"✓ {custom_id}: Added {len(batch_cats)} categories")
                    else:
                        print(f"⚠ {custom_id}: Expected {current_batch_size} categories, got {len(batch_cats)}")
                        # Take what we have and fill the rest
                        split_categories.extend(batch_cats[:current_batch_size])
                        if len(batch_cats) < current_batch_size:
                            missing = current_batch_size - len(batch_cats)
                            split_categories.extend(['failed_parsing'] * missing)
                else:
                    print(f"✗ {custom_id}: Not found in responses")
                    split_categories.extend(['failed'] * current_batch_size)
                    failed_batches.add(custom_id)
            else:
                # No more custom_ids available
                print(f"✗ No more batch responses available - adding {current_batch_size} 'failed' entries")
                split_categories.extend(['failed'] * current_batch_size)

            samples_processed += current_batch_size

        # Validation
        print(f"Generated {len(split_categories)} categories for {num_samples} samples")

        if len(split_categories) != num_samples:
            print(f"ERROR: Category count mismatch!")
            print(f"Expected: {num_samples}, Got: {len(split_categories)}")

            if len(split_categories) < num_samples:
                missing = num_samples - len(split_categories)
                print(f"Adding {missing} 'missing' entries")
                split_categories.extend(['missing'] * missing)
            elif len(split_categories) > num_samples:
                print(f"Truncating to {num_samples} entries")
                split_categories = split_categories[:num_samples]

        # Count categories
        category_counts = {}
        for cat in split_categories:
            category_counts[cat] = category_counts.get(cat, 0) + 1

        print(f"Category distribution for {split_name}:")
        for cat, count in sorted(category_counts.items()):
            print(f"  {cat}: {count}")

        # Add category column
        try:
            dataset_with_categories = dataset.add_column('category', split_categories)
            updated_splits[split_name] = dataset_with_categories
            print(f"✓ Successfully added categories to {split_name}")
        except Exception as e:
            print(f"✗ Error adding categories to {split_name}: {e}")
            raise

    # Create DatasetDict
    dataset_dict = DatasetDict(updated_splits)

    # Step 3: Apply filtering if requested
    if filter_failed:
        print("\nApplying filtering to remove failed entries...")
        dataset_dict = filter_failed_parsing_datasetdict(dataset_dict)

    # Final summary
    print(f"\nSUMMARY:")
    for split_name, split_dataset in dataset_dict.items():
        print(f"{split_name}: {len(split_dataset)} samples")

    if failed_batches:
        print(f"Failed batches: {sorted(failed_batches)}")

    return dataset_dict


def parse_batch_content(batch_content):
    """
    Robust parser for batch content (JSON/JSONL format)
    """
    import json

    if not batch_content:
        print("Empty batch content received")
        return []

    # Handle different input types
    if hasattr(batch_content, 'text'):
        batch_content = batch_content.text
    elif not isinstance(batch_content, str):
        batch_content = str(batch_content)

    batch_content = batch_content.strip()

    if not batch_content:
        print("Empty batch content after processing")
        return []

    print(f"Content length: {len(batch_content)} characters")
    print(f"Content starts with: {repr(batch_content[:100])}")
    print(f"Content ends with: {repr(batch_content[-100:])}")

    responses = []

    try:
        # Method 1: Try JSONL format (most common)
        print("Attempting JSONL parsing...")
        lines = batch_content.split('\n')
        print(f"Found {len(lines)} lines")

        for i, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue

            try:
                response = json.loads(line)

                # Validate response structure
                if 'custom_id' not in response:
                    print(f"Line {i+1}: Missing custom_id")
                    continue

                if 'response' not in response:
                    print(f"Line {i+1}: Missing response field")
                    continue

                # Check for error field
                if response.get('error'):
                    print(f"Line {i+1}: Response has error: {response['error']}")
                    continue

                # Validate nested structure
                try:
                    content = response['response']['body']['choices'][0]['message']['content']
                    responses.append(response)

                    if i < 5:  # Show first few for debugging
                        print(f"✓ Line {i+1}: {response['custom_id']} -> '{content}'")

                except (KeyError, IndexError, TypeError) as e:
                    print(f"Line {i+1}: Invalid response structure: {e}")
                    continue

            except json.JSONDecodeError as e:
                print(f"Line {i+1}: JSON decode error: {e}")
                if len(line) < 200:
                    print(f"  Full line: {repr(line)}")
                else:
                    print(f"  Line preview: {repr(line[:100])}...{repr(line[-100:])}")
                continue

        if responses:
            print(f"Successfully parsed {len(responses)} responses from JSONL")
            return responses

        # Method 2: Try JSON array format
        print("JSONL failed, attempting JSON array parsing...")
        if batch_content.startswith('[') and batch_content.endswith(']'):
            data = json.loads(batch_content)
            if isinstance(data, list):
                print(f"Successfully parsed {len(data)} responses from JSON array")
                return data

        # Method 3: Try single JSON object
        print("Attempting single JSON object parsing...")
        data = json.loads(batch_content)
        if isinstance(data, dict):
            if 'responses' in data:
                return data['responses']
            elif 'data' in data:
                return data['data']
            else:
                return [data]

    except Exception as e:
        print(f"All parsing methods failed: {e}")

    return []


def extract_category_from_text(text):
    """
    Extract category from text, handling various formats
    """
    import re

    if not text or not isinstance(text, str):
        return None

    text = text.strip()
    if not text:
        return None

    # Handle numbered format: "1. category" or "2. system_leak"
    numbered_match = re.match(r'^\d+\.\s*(.+)', text)
    if numbered_match:
        category = numbered_match.group(1).strip()
    # Handle bullet formats: "- category" or "* category"
    elif text.startswith(('- ', '* ')):
        category = text[2:].strip()
    # Handle colon format: "Category: value"
    elif ':' in text:
        category = text.split(':', 1)[1].strip()
    else:
        category = text

    # Clean formatting
    category = re.sub(r'[*`"\'()[\]{}]', '', category).strip()

    # Remove common prefixes
    prefixes = ['category', 'type', 'classification', 'label', 'answer', 'result']
    category_lower = category.lower()
    for prefix in prefixes:
        if category_lower.startswith(prefix + ':'):
            category = category[len(prefix)+1:].strip()
            break
        elif category_lower.startswith(prefix + ' '):
            category = category[len(prefix)+1:].strip()
            break

    # Convert to lowercase
    category = category.lower().strip()

    return category if category else None


def is_valid_category(category):
    """
    Check if category is valid
    """
    if not category or len(category) < 3:
        return False

    valid_categories = {
        'jailbreak',
        'system_leak',
        'task_override',
        'encoding_manipulation',
        'prompt_confusion'
    }

    return category.lower() in valid_categories


def filter_failed_parsing_datasetdict(dataset_dict):
    """
    Filter out failed entries from DatasetDict
    """
    from datasets import DatasetDict

    filtered_dict = {}
    failure_types = {'failed', 'failed_parsing', 'missing'}

    for phase_name, dataset in dataset_dict.items():
        print(f"\nFiltering {phase_name}...")

        # Show before filtering
        before_count = len(dataset)
        category_counts = {}
        for example in dataset:
            cat = example['category']
            category_counts[cat] = category_counts.get(cat, 0) + 1

        print(f"Before filtering ({before_count} samples):")
        for cat, count in sorted(category_counts.items()):
            print(f"  {cat}: {count}")

        # Filter out failure types
        filtered_dataset = dataset.filter(lambda example: example['category'] not in failure_types)
        filtered_dict[phase_name] = filtered_dataset

        # Show after filtering
        after_count = len(filtered_dataset)
        print(f"After filtering: {before_count} -> {after_count} ({before_count - after_count} removed)")

    return DatasetDict(filtered_dict)

In [ ]:
updated_dataset_part_one = process_batch_and_add_categories(
     original_dataset=dataset_injected_first,
     batch_content=file_response.text,
     batch_size=1
 )


In [ ]:
updated_dataset_part_two1 = process_batch_and_add_categories(
     original_dataset=dataset_injected_second1,
     batch_content=file_response_second1.text,
     batch_size=1
 )


In [ ]:
updated_dataset_part_two2 = process_batch_and_add_categories(
     original_dataset=dataset_injected_second2,
     batch_content=file_response_second2.text,
     batch_size=1
 )



In [ ]:
from datasets import concatenate_datasets
phase1 = concatenate_datasets([updated_dataset_part_two1["Phase1"], updated_dataset_part_two2["Phase1"]])
phase2 = concatenate_datasets([updated_dataset_part_two1["Phase2"], updated_dataset_part_two2["Phase2"]])

updated_dataset_part_two = DatasetDict({"Phase1": phase1, "Phase2":phase2})
updated_dataset_part_two

In [ ]:
updated_dataset_part_three = process_batch_and_add_categories(
     original_dataset=dataset_injected_third,
     batch_content=file_response_third.text,
     batch_size=1
 )

In [ ]:
updated_dataset_part_four = process_batch_and_add_categories(
     original_dataset=dataset_injected_fourth,
     batch_content=file_response_fourth.text,
     batch_size=1
 )


In [ ]:
from datasets import concatenate_datasets

updated_dataset_full_phase1 = concatenate_datasets([updated_dataset_part_one["Phase1"], updated_dataset_part_two["Phase1"], updated_dataset_part_three["Phase1"], updated_dataset_part_four["Phase1"]])
updated_dataset_full_phase2 = concatenate_datasets([updated_dataset_part_one["Phase2"], updated_dataset_part_two["Phase2"], updated_dataset_part_three["Phase2"], updated_dataset_part_four["Phase2"]])

updated_dataset_full = DatasetDict({"Phase1":updated_dataset_full_phase1, "Phase2":updated_dataset_full_phase2})


In [ ]:
updated_dataset_full

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
import json
import os

output_dir = '/content/drive/MyDrive/Algoverse/'
os.makedirs(output_dir, exist_ok=True)


json_data = {}

for phase_name, dataset in updated_dataset_full.items():
    print(f"Processing {phase_name}...")


    phase_data = []
    for i in range(len(dataset)):
        row = {}
        for feature in dataset.features:
            row[feature] = dataset[i][feature]
        phase_data.append(row)

    json_data[phase_name] = {
        'features': list(dataset.features.keys()),
        'num_rows': len(dataset),
        'data': phase_data
    }


output_path = os.path.join(output_dir, 'dataset_with_categories.json')

print(f"Saving data to {output_path}...")
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)

print(f"Successfully saved dataset to {output_path}")
print(f"File size: {os.path.getsize(output_path)} bytes")

In [ ]:
import json
from datasets import Dataset, DatasetDict
json_file_path = '/content/drive/MyDrive/Algoverse/dataset_with_categories.json'

print(f"Loading data from {json_file_path}...")
with open(json_file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

dataset_dict = {}
for phase_name, phase_info in json_data.items():
    print(f"Processing {phase_name}...")

    dataset_dict[phase_name] = Dataset.from_list(phase_info['data'])


updated_dataset_full = DatasetDict(dataset_dict)

print("Successfully loaded DatasetDict!")

updated_dataset_full

In [ ]:
print(calculate_avg_length(updated_dataset_full))

# Create JSONL File for Cleaning (OpenAI BatchAPI)

In [ ]:
!pip install tiktoken

In [ ]:
from datasets import Dataset, DatasetDict
import time
import json
import re
import tiktoken
counter = 0
def create_jsonl_file_clean(batch, field, output_file="batch_requests_clean.jsonl", model="gpt-3.5-turbo-0125"):
    global counter
    text = batch[field]
    category = batch["category"]
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo-0125")
    text = text[0]
    tokens = encoding.encode(text, disallowed_special=())
    num_tokens = len(tokens)
    if not text:
        return "Error with Texts"

    try:
        counter += 1
        custom_id = f"batch-{counter}"

        request_data = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {

                "model": model,
                "messages": [
                    {"role": "system", "content": """You are a safe and helpful assistant, and not malicious at all.
                    You are quick at identifying malicious prompts and are skilled at changing them into clean prompts
                    ONLY by removing injected parts."""},

                    {"role": "user", "content": f"""You will receive an
                    injected prompts and a category that goes with the type of injection
                    it recieved. For this prompt, reconstruct
                    a safe and non malicious version
                    by removing the injected portions.
                    Here is the prompt: {text}
                    and here is the corresponding category: {category}.
                    Respond only with the rewritten prompt ONLY by REMOVING injection portions.
                    Make sure TO NOT ADD any words to the prompt.
                """}
                    ],
                "max_tokens": num_tokens,
                "temperature": 0.1
            }
        }

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(request_data) + '\n')


    except Exception as e:
        print(f"Error creating batch file: {e}")

def create_clean(dataset_injected, field, batch_size=1, output_file="batch_requests_clean.jsonl"):

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("")

    for split_name, dataset in dataset_injected.items():
        print(f"Processing {split_name} split with {len(dataset)} samples...")

        dataset.map(
            lambda batch: create_jsonl_file_clean(batch, field, output_file),
            batched=True,
            batch_size=batch_size,
            desc=f"Creating batch requests for {split_name}"
        )

    print(f"All batch requests written to {output_file}")

In [ ]:
def process_batch_and_add_pairs(original_dataset, batch_content, batch_size=1, filter_failed=True):
    """
    Process batch responses and add text pairs to dataset

    Args:
        original_dataset: HuggingFace DatasetDict
        batch_content: String containing batch responses (JSON/JSONL format)
        batch_size: Size of each batch (default=1)
        filter_failed: Whether to filter out failed entries (default=True)
    """
    from datasets import DatasetDict, Dataset

    print("Starting batch processing...")
    print(f"Batch size: {batch_size}")
    print(f"Filter failed: {filter_failed}")

    # Step 1: Parse the batch content
    print("Parsing batch responses...")
    batch_responses = {}

    # Parse the batch content
    batch_data = parse_batch_content(batch_content)

    if not batch_data:
        print("ERROR: No batch data could be parsed from the input!")
        print(f"Input content preview: {repr(batch_content[:500] if batch_content else 'None')}...")
        return original_dataset

    print(f"Successfully parsed {len(batch_data)} batch responses")

    # Process each response
    for response_data in batch_data:
        try:
            custom_id = response_data['custom_id']

            # Extract content from response
            content = response_data['response']['body']['choices'][0]['message']['content']

            # Extract and clean the text pairs
            text_pairs = []

            # For batch_size=1, we expect a single text response
            if batch_size == 1:
                cleaned_text = extract_and_clean_text(content)
                if cleaned_text:
                    text_pairs = [cleaned_text]
                else:
                    print(f"Empty or invalid text extracted from {custom_id}")
                    text_pairs = ['failed_extraction']
            else:
                # Handle multiple text pairs (split by lines or other delimiters)
                content_lines = content.split('\n')

                for line_content in content_lines:
                    cleaned_text = extract_and_clean_text(line_content)
                    if cleaned_text:
                        text_pairs.append(cleaned_text)

                # Validate we have the expected number of text pairs
                if len(text_pairs) != batch_size:
                    print(f"Expected {batch_size} text pairs for {custom_id}, but found {len(text_pairs)}")
                    print(f"Text pairs found: {len(text_pairs)}")
                    print(f"Raw content: {repr(content)}")

                    # Adjust text pairs list
                    if len(text_pairs) < batch_size:
                        missing = batch_size - len(text_pairs)
                        text_pairs.extend(['failed_extraction'] * missing)
                    else:
                        text_pairs = text_pairs[:batch_size]

            batch_responses[custom_id] = text_pairs
            print(f"Batch {custom_id}: Found {len(text_pairs)} text pairs")

        except Exception as e:
            print(f"Error parsing batch response: {e}")
            print(f"Response data keys: {list(response_data.keys()) if isinstance(response_data, dict) else 'Not a dict'}")
            if isinstance(response_data, dict):
                print(f"Custom ID: {response_data.get('custom_id', 'MISSING')}")
            continue

    print(f"Found {len(batch_responses)} successful batches")

    # Step 2: Process each split
    updated_splits = {}
    failed_batches = set()

    # First, let's see what custom_ids we actually have
    available_custom_ids = sorted(batch_responses.keys())
    print(f"Available custom_ids: {available_custom_ids[:10]}..." if len(available_custom_ids) > 10 else f"Available custom_ids: {available_custom_ids}")

    # Extract just the batch numbers to understand the sequence
    batch_numbers = []
    for custom_id in available_custom_ids:
        try:
            parts = custom_id.split('-')
            if len(parts) >= 2:
                batch_num = int(parts[1])
                batch_numbers.append(batch_num)
        except ValueError:
            continue

    if batch_numbers:
        print(f"Batch number range: {min(batch_numbers)} to {max(batch_numbers)} ({len(batch_numbers)} total)")

    for split_name, dataset in original_dataset.items():
        print(f"\nProcessing {split_name}...")

        split_pairs = []
        num_samples = len(dataset)

        print(f"Dataset has {num_samples} samples")

        # Calculate expected number of batches for this split
        expected_batches = (num_samples + batch_size - 1) // batch_size  # Ceiling division
        print(f"Expected {expected_batches} batches for {num_samples} samples with batch_size={batch_size}")

        # Extract batch numbers from available responses to understand the numbering scheme
        available_batch_nums = []
        for custom_id in batch_responses.keys():
            try:
                parts = custom_id.split('-')
                if len(parts) >= 2 and parts[1].isdigit():
                    batch_num = int(parts[1])
                    available_batch_nums.append(batch_num)
            except:
                continue

        available_batch_nums.sort()

        if available_batch_nums:
            print(f"Available batch numbers: {available_batch_nums[0]} to {available_batch_nums[-1]} ({len(available_batch_nums)} total)")
            start_batch_num = available_batch_nums[0]
            end_batch_num = available_batch_nums[-1]
            expected_end_batch = start_batch_num + expected_batches - 1
            print(f"Expected batch range for this dataset: {start_batch_num} to {expected_end_batch}")
        else:
            print("No valid batch numbers found in responses")
            split_pairs = ['api_failed'] * num_samples
            failed_batches.update([f"no-batch-nums"])
            continue

        # Process samples in order, looking for the corresponding batch numbers
        split_pairs = []

        for i in range(0, num_samples, batch_size):
            current_batch_size = min(batch_size, num_samples - i)
            dataset_batch_index = i // batch_size  # 0, 1, 2, 3, ...

            # Calculate expected batch number for this dataset position
            expected_batch_num = start_batch_num + dataset_batch_index
            expected_custom_id = f"batch-{expected_batch_num}"

            # Look for this specific batch
            if expected_custom_id in batch_responses:
                batch_pairs = batch_responses[expected_custom_id]

                # Ensure we have the right number of text pairs
                if len(batch_pairs) == current_batch_size:
                    split_pairs.extend(batch_pairs)
                    print(f"✓ Dataset position {dataset_batch_index} -> {expected_custom_id}: Added {len(batch_pairs)} text pairs")
                else:
                    print(f"⚠ Dataset position {dataset_batch_index} -> {expected_custom_id}: Expected {current_batch_size} text pairs, got {len(batch_pairs)}")
                    # Take what we have and fill the rest
                    split_pairs.extend(batch_pairs[:current_batch_size])
                    if len(batch_pairs) < current_batch_size:
                        missing = current_batch_size - len(batch_pairs)
                        split_pairs.extend(['failed_extraction'] * missing)
            else:
                # This specific batch is missing (failed at API level)
                print(f"✗ Dataset position {dataset_batch_index} -> {expected_custom_id}: Missing - marking as api_failed")
                split_pairs.extend(['api_failed'] * current_batch_size)
                failed_batches.add(expected_custom_id)

        # Validation
        print(f"Generated {len(split_pairs)} text pairs for {num_samples} samples")

        if len(split_pairs) != num_samples:
            print(f"ERROR: Text pair count mismatch!")
            print(f"Expected: {num_samples}, Got: {len(split_pairs)}")

            if len(split_pairs) < num_samples:
                missing = num_samples - len(split_pairs)
                print(f"Adding {missing} 'missing' entries")
                split_pairs.extend(['missing'] * missing)
            elif len(split_pairs) > num_samples:
                print(f"Truncating to {num_samples} entries")
                split_pairs = split_pairs[:num_samples]

        # Count text pair types
        pair_counts = {}
        for pair in split_pairs:
            if pair in ['failed', 'failed_extraction', 'missing', 'api_failed']:
                pair_type = pair
            else:
                pair_type = 'valid_text'
            pair_counts[pair_type] = pair_counts.get(pair_type, 0) + 1

        print(f"Text pair distribution for {split_name}:")
        for pair_type, count in sorted(pair_counts.items()):
            print(f"  {pair_type}: {count}")

        # Add pair column
        try:
            dataset_with_pairs = dataset.add_column('pair', split_pairs)
            updated_splits[split_name] = dataset_with_pairs
            print(f"✓ Successfully added text pairs to {split_name}")
        except Exception as e:
            print(f"✗ Error adding text pairs to {split_name}: {e}")
            raise

    # Create DatasetDict
    dataset_dict = DatasetDict(updated_splits)

    # Step 3: Apply filtering if requested
    if filter_failed:
        print("\nApplying filtering to remove failed entries...")
        dataset_dict = filter_failed_extraction_datasetdict(dataset_dict)

    # Final summary
    print(f"\nSUMMARY:")
    for split_name, split_dataset in dataset_dict.items():
        print(f"{split_name}: {len(split_dataset)} samples")

    if failed_batches:
        print(f"Failed batches: {sorted(failed_batches)}")

    return dataset_dict


def parse_batch_content(batch_content):
    """
    Robust parser for batch content (JSON/JSONL format)
    """
    import json

    if not batch_content:
        print("Empty batch content received")
        return []

    # Handle different input types
    if hasattr(batch_content, 'text'):
        batch_content = batch_content.text
    elif not isinstance(batch_content, str):
        batch_content = str(batch_content)

    batch_content = batch_content.strip()

    if not batch_content:
        print("Empty batch content after processing")
        return []

    print(f"Content length: {len(batch_content)} characters")
    print(f"Content starts with: {repr(batch_content[:100])}")
    print(f"Content ends with: {repr(batch_content[-100:])}")

    responses = []

    try:
        # Method 1: Try JSONL format (most common)
        print("Attempting JSONL parsing...")
        lines = batch_content.split('\n')
        print(f"Found {len(lines)} lines")

        for i, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue

            try:
                response = json.loads(line)

                # Validate response structure
                if 'custom_id' not in response:
                    print(f"Line {i+1}: Missing custom_id")
                    continue

                if 'response' not in response:
                    print(f"Line {i+1}: Missing response field")
                    continue

                # Check for error field
                if response.get('error'):
                    print(f"Line {i+1}: Response has error: {response['error']}")
                    continue

                # Validate nested structure
                try:
                    content = response['response']['body']['choices'][0]['message']['content']
                    responses.append(response)

                    if i < 5:  # Show first few for debugging
                        print(f"✓ Line {i+1}: {response['custom_id']} -> '{content[:100]}...' ({len(content)} chars)")

                except (KeyError, IndexError, TypeError) as e:
                    print(f"Line {i+1}: Invalid response structure: {e}")
                    continue

            except json.JSONDecodeError as e:
                print(f"Line {i+1}: JSON decode error: {e}")
                if len(line) < 200:
                    print(f"  Full line: {repr(line)}")
                else:
                    print(f"  Line preview: {repr(line[:100])}...{repr(line[-100:])}")
                continue

        if responses:
            print(f"Successfully parsed {len(responses)} responses from JSONL")
            return responses

        # Method 2: Try JSON array format
        print("JSONL failed, attempting JSON array parsing...")
        if batch_content.startswith('[') and batch_content.endswith(']'):
            data = json.loads(batch_content)
            if isinstance(data, list):
                print(f"Successfully parsed {len(data)} responses from JSON array")
                return data

        # Method 3: Try single JSON object
        print("Attempting single JSON object parsing...")
        data = json.loads(batch_content)
        if isinstance(data, dict):
            if 'responses' in data:
                return data['responses']
            elif 'data' in data:
                return data['data']
            else:
                return [data]

    except Exception as e:
        print(f"All parsing methods failed: {e}")

    return []


def extract_and_clean_text(text):
    """
    Extract and clean text content, preserving the full text but removing extra whitespace
    """
    if not text or not isinstance(text, str):
        return None

    # Strip leading and trailing whitespace
    text = text.strip()

    if not text:
        return None

    # Replace multiple consecutive whitespace characters (including newlines) with single spaces
    import re
    cleaned_text = re.sub(r'\s+', ' ', text)

    # Final strip to ensure no leading/trailing spaces
    cleaned_text = cleaned_text.strip()

    return cleaned_text if cleaned_text else None


def filter_failed_extraction_datasetdict(dataset_dict):
    """
    Filter out failed entries from DatasetDict
    """
    from datasets import DatasetDict

    filtered_dict = {}
    failure_types = {'failed', 'failed_extraction', 'missing', 'api_failed'}

    for phase_name, dataset in dataset_dict.items():
        print(f"\nFiltering {phase_name}...")

        # Show before filtering
        before_count = len(dataset)
        pair_counts = {}
        for example in dataset:
            pair = example['pair']
            pair_type = 'valid_text' if pair not in failure_types else pair
            pair_counts[pair_type] = pair_counts.get(pair_type, 0) + 1

        print(f"Before filtering ({before_count} samples):")
        for pair_type, count in sorted(pair_counts.items()):
            print(f"  {pair_type}: {count}")

        # Filter out failure types
        filtered_dataset = dataset.filter(lambda example: example['pair'] not in failure_types)
        filtered_dict[phase_name] = filtered_dataset

        # Show after filtering
        after_count = len(filtered_dataset)
        print(f"After filtering: {before_count} -> {after_count} ({before_count - after_count} removed)")

    return DatasetDict(filtered_dict)

# Use OpenAI Batch API to create clean dataset

In [ ]:
dataset_injected_first, dataset_injected_second, dataset_injected_third, dataset_injected_fourth = split_dataset_dict(updated_dataset_full)

In [ ]:
dataset_injected_second1, dataset_injected_second2 = split_dataset_dict_half(dataset_injected_second)

In [ ]:
create_clean(dataset_injected_first, "body", output_file="batch_requests_clean_first.jsonl")

In [ ]:
batch_input_file = client.files.create(
    file=open("batch_requests_clean_first.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch_input_file_id = batch_input_file.id
batch_val = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "create clean first"
    }
)
print(batch_val)

In [ ]:
batch = client.batches.retrieve("batch_68881ad42820819097ab5632a4bea1dc")
print(batch)
batch_output_file_id = batch.output_file_id

In [ ]:
# Retrieve the batch using the ID you already have
batch = client.batches.retrieve("batch_68881ad42820819097ab5632a4bea1dc")
print(f"Batch status: {batch.status}")

# Check if the batch is completed
if batch.status == "completed":
    batch_output_file_id = batch.output_file_id
    print(f"batch_output_file_id = {batch_output_file_id}")

    # Now you can get the file content
    file_response = client.files.content(batch_output_file_id)
    print(file_response.text)

elif batch.status == "failed":
    print("Batch failed!")
    print(f"Error details: {batch}")

else:
    print(f"Batch is still {batch.status}. Please wait and try again.")

NameError: name 'client' is not defined

In [ ]:
file_response = client.files.content(batch_output_file_id)
print(file_response.text)

In [ ]:
create_clean(dataset_injected_second1, "body", batch_size=1, output_file="batch_requests_clean_second1.jsonl")

In [ ]:
batch_input_file_second1 = client.files.create(
    file=open("batch_requests_clean_second1.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_second1)

In [ ]:
batch_input_file_id_second1 = batch_input_file_second1.id
batch_val_second1 = client.batches.create(
    input_file_id=batch_input_file_id_second1,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "create clean second part 1"
    }
)

In [ ]:
batch_second1 = client.batches.retrieve("batch_6888055fdfa88190b73f315cff6bf4d3")
print(batch_second1)
batch_output_file_id_second1 = batch_second1.output_file_id

In [ ]:
file_response_second1 = client.files.content(batch_output_file_id_second1)
print(file_response_second1.text)

In [ ]:
create_clean(dataset_injected_second2, "body", output_file="batch_requests_clean_second2.jsonl")

In [ ]:
batch_input_file_second2 = client.files.create(
    file=open("batch_requests_clean_second2.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_second2)

In [ ]:
batch_input_file_id_second2 = batch_input_file_second2.id
batch_val_second2 = client.batches.create(
    input_file_id=batch_input_file_id_second2,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "create clean second part 2"
    }
)

In [ ]:
batch_second2 = client.batches.retrieve("batch_6888058ed9a88190b564a1ccec6ec333")
print(batch_second2)
batch_output_file_id_second2 = batch_second2.output_file_id

In [ ]:
file_response_second2 = client.files.content(batch_output_file_id_second2)
print(file_response_second2.text)

In [ ]:
create_clean(dataset_injected_third, "body", output_file="batch_requests_clean_third.jsonl")

In [ ]:
batch_input_file_third = client.files.create(
    file=open("batch_requests_clean_third.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_third)

In [ ]:
batch_input_file_id_third = batch_input_file_third.id
batch_val_third = client.batches.create(
    input_file_id=batch_input_file_id_third,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "create clean part three"
    }
)

In [ ]:
batch_third = client.batches.retrieve("batch_6888063b192081908f0cca7300a3e217")
print(batch_third)
batch_output_file_id_third = batch_third.output_file_id

In [ ]:
file_response_third = client.files.content(batch_output_file_id_third)
# print(file_response_third.text)

In [ ]:
create_clean(dataset_injected_fourth, "body", output_file="batch_requests_clean_fourth.jsonl")

In [ ]:
batch_input_file_fourth = client.files.create(
    file=open("batch_requests_clean_fourth.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file_fourth)

In [ ]:
batch_input_file_id_fourth = batch_input_file_fourth.id
batch_val_fourth = client.batches.create(
    input_file_id=batch_input_file_id_fourth,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "create clean part four"
    }
)

In [ ]:
batch_fourth = client.batches.retrieve("batch_688806be4608819087417496e9253385")
print(batch_fourth)
batch_output_file_id_fourth = batch_fourth.output_file_id

In [ ]:
file_response_fourth = client.files.content(batch_output_file_id_fourth)
# print(file_response_fourth.text)

In [ ]:
updated_dataset_part_one_clean = process_batch_and_add_pairs(
     original_dataset=dataset_injected_first,
     batch_content=file_response.text,
     batch_size=1
 )


In [ ]:
updated_dataset_part_two1_clean = process_batch_and_add_pairs(
     original_dataset=dataset_injected_second1,
     batch_content=file_response_second1.text,
     batch_size=1
 )


In [ ]:
updated_dataset_part_two2_clean = process_batch_and_add_pairs(
     original_dataset=dataset_injected_second2,
     batch_content=file_response_second2.text,
     batch_size=1
 )



In [ ]:
from datasets import concatenate_datasets
phase1 = concatenate_datasets([updated_dataset_part_two1_clean["Phase1"], updated_dataset_part_two2_clean["Phase1"]])
phase2 = concatenate_datasets([updated_dataset_part_two1_clean["Phase2"], updated_dataset_part_two2_clean["Phase2"]])

updated_dataset_part_two_clean = DatasetDict({"Phase1": phase1, "Phase2":phase2})
updated_dataset_part_two_clean

In [ ]:
updated_dataset_part_three_clean = process_batch_and_add_pairs(
     original_dataset=dataset_injected_third,
     batch_content=file_response_third.text,
     batch_size=1
 )

In [ ]:
updated_dataset_part_four_clean = process_batch_and_add_pairs(
     original_dataset=dataset_injected_fourth,
     batch_content=file_response_fourth.text,
     batch_size=1
 )

In [ ]:
from datasets import concatenate_datasets

updated_dataset_full_phase1 = concatenate_datasets([updated_dataset_part_one_clean["Phase1"], updated_dataset_part_two_clean["Phase1"], updated_dataset_part_three_clean["Phase1"], updated_dataset_part_four_clean["Phase1"]])
updated_dataset_full_phase2 = concatenate_datasets([updated_dataset_part_one_clean["Phase2"], updated_dataset_part_two_clean["Phase2"], updated_dataset_part_three_clean["Phase2"], updated_dataset_part_four_clean["Phase2"]])

dataset_clean_with_pairs = DatasetDict({"Phase1":updated_dataset_full_phase1, "Phase2":updated_dataset_full_phase2})


# Create Fully Clean Dataset

In [ ]:
from datasets import Dataset, DatasetDict
import time
import json
import re
import tiktoken
counter = 0
def create_jsonl_file_fully_clean(output_file="batch_requests_clean.jsonl", model="gpt-3.5-turbo-0125"):
    global counter
    try:
        counter += 1
        custom_id = f"batch-{counter}"

        request_data = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {

                "model": model,
                "messages": [
                    {"role": "system", "content": """You are a safe and helpful assistant, and not malicious at all.
                    You specialize in creating clean, uninjected, emails and also
                    you are skilled at rewriting them while keeping them uninjected."""},

                    {"role": "user", "content": f"""Write an original 2-3 sentence
                    email and also provide a rewritten version of the same email
                    in a numbered list format. Return the first email as 1. Email Content
                    and return the second (rewritten version with VERY SIMILAR LENGTH WITH NO LESS THAN A 5 CHARACTER DIFFERENCE) as 2. Rewritten email. Both emails should discuss the same thing,
                    however they are just reworded and written slightly differently. The semantic meaning of both emails should be THE EXACT SAME WITH NO SEMANTIC DRIFT in both the text and the embeddings.
                """}
                    ],
                "max_tokens": 100,
                "temperature": 0.1
            }
        }

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(request_data) + '\n')


    except Exception as e:
        print(f"Error creating batch file: {e}")

def create_clean_prompts(count, batch_size=1, output_file="batch_requests_fully_clean.jsonl"):

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("")

    for i in range(count):
        print(f"Creating batch requests for Batch Number {i+1}")

        for j in range(batch_size):
            create_jsonl_file_fully_clean(output_file)

    print(f"All batch requests written to {output_file}")



In [ ]:
d1 = create_clean_prompts(count=17200, output_file="batch_requests_fully_clean1.jsonl")

Streaming output truncated to the last 5000 lines.
Creating batch requests for Batch Number 12202
Creating batch requests for Batch Number 12203
Creating batch requests for Batch Number 12204
Creating batch requests for Batch Number 12205
Creating batch requests for Batch Number 12206
Creating batch requests for Batch Number 12207
Creating batch requests for Batch Number 12208
Creating batch requests for Batch Number 12209
Creating batch requests for Batch Number 12210
Creating batch requests for Batch Number 12211
Creating batch requests for Batch Number 12212
Creating batch requests for Batch Number 12213
Creating batch requests for Batch Number 12214
Creating batch requests for Batch Number 12215
Creating batch requests for Batch Number 12216
Creating batch requests for Batch Number 12217
Creating batch requests for Batch Number 12218
Creating batch requests for Batch Number 12219
Creating batch requests for Batch Number 12220
Creating batch requests for Batch Number 12221
Creating 

In [ ]:
d2 = create_clean_prompts(count=17200, output_file="batch_requests_fully_clean2.jsonl")

Streaming output truncated to the last 5000 lines.
Creating batch requests for Batch Number 12202
Creating batch requests for Batch Number 12203
Creating batch requests for Batch Number 12204
Creating batch requests for Batch Number 12205
Creating batch requests for Batch Number 12206
Creating batch requests for Batch Number 12207
Creating batch requests for Batch Number 12208
Creating batch requests for Batch Number 12209
Creating batch requests for Batch Number 12210
Creating batch requests for Batch Number 12211
Creating batch requests for Batch Number 12212
Creating batch requests for Batch Number 12213
Creating batch requests for Batch Number 12214
Creating batch requests for Batch Number 12215
Creating batch requests for Batch Number 12216
Creating batch requests for Batch Number 12217
Creating batch requests for Batch Number 12218
Creating batch requests for Batch Number 12219
Creating batch requests for Batch Number 12220
Creating batch requests for Batch Number 12221
Creating 

In [ ]:
d3 = create_clean_prompts(count=17200, output_file="batch_requests_fully_clean3.jsonl")

Streaming output truncated to the last 5000 lines.
Creating batch requests for Batch Number 12202
Creating batch requests for Batch Number 12203
Creating batch requests for Batch Number 12204
Creating batch requests for Batch Number 12205
Creating batch requests for Batch Number 12206
Creating batch requests for Batch Number 12207
Creating batch requests for Batch Number 12208
Creating batch requests for Batch Number 12209
Creating batch requests for Batch Number 12210
Creating batch requests for Batch Number 12211
Creating batch requests for Batch Number 12212
Creating batch requests for Batch Number 12213
Creating batch requests for Batch Number 12214
Creating batch requests for Batch Number 12215
Creating batch requests for Batch Number 12216
Creating batch requests for Batch Number 12217
Creating batch requests for Batch Number 12218
Creating batch requests for Batch Number 12219
Creating batch requests for Batch Number 12220
Creating batch requests for Batch Number 12221
Creating 

In [ ]:
d4 = create_clean_prompts(count=17200, output_file="batch_requests_fully_clean4.jsonl")

Streaming output truncated to the last 5000 lines.
Creating batch requests for Batch Number 12202
Creating batch requests for Batch Number 12203
Creating batch requests for Batch Number 12204
Creating batch requests for Batch Number 12205
Creating batch requests for Batch Number 12206
Creating batch requests for Batch Number 12207
Creating batch requests for Batch Number 12208
Creating batch requests for Batch Number 12209
Creating batch requests for Batch Number 12210
Creating batch requests for Batch Number 12211
Creating batch requests for Batch Number 12212
Creating batch requests for Batch Number 12213
Creating batch requests for Batch Number 12214
Creating batch requests for Batch Number 12215
Creating batch requests for Batch Number 12216
Creating batch requests for Batch Number 12217
Creating batch requests for Batch Number 12218
Creating batch requests for Batch Number 12219
Creating batch requests for Batch Number 12220
Creating batch requests for Batch Number 12221
Creating 

In [ ]:
d5 = create_clean_prompts(count=17200, output_file="batch_requests_fully_clean5.jsonl")

Streaming output truncated to the last 5000 lines.
Creating batch requests for Batch Number 12202
Creating batch requests for Batch Number 12203
Creating batch requests for Batch Number 12204
Creating batch requests for Batch Number 12205
Creating batch requests for Batch Number 12206
Creating batch requests for Batch Number 12207
Creating batch requests for Batch Number 12208
Creating batch requests for Batch Number 12209
Creating batch requests for Batch Number 12210
Creating batch requests for Batch Number 12211
Creating batch requests for Batch Number 12212
Creating batch requests for Batch Number 12213
Creating batch requests for Batch Number 12214
Creating batch requests for Batch Number 12215
Creating batch requests for Batch Number 12216
Creating batch requests for Batch Number 12217
Creating batch requests for Batch Number 12218
Creating batch requests for Batch Number 12219
Creating batch requests for Batch Number 12220
Creating batch requests for Batch Number 12221
Creating 

In [ ]:
def create_batch_job(client, file_path, description="batch job", endpoint="/v1/chat/completions", completion_window="24h"):
    try:
        batch_input_file = client.files.create(
            file=open(file_path, "rb"),
            purpose="batch"
        )
        print(f"Uploaded file: {batch_input_file}")

        batch_val = client.batches.create(
            input_file_id=batch_input_file.id,
            endpoint=endpoint,
            completion_window=completion_window,
            metadata={"description": description}
        )
        print(f"Created batch: {batch_val}")

        return batch_val.id

    except Exception as e:
        print(f"Error creating batch job: {e}")
        return None


def check_batch_status(client, batch_id):
    try:
        batch = client.batches.retrieve(batch_id)
        print(f"Batch status: {batch}")

        result = {
            "batch_id": batch_id,
            "status": batch.status,
            "created_at": batch.created_at,
            "completed_at": getattr(batch, 'completed_at', None),
            "failed_at": getattr(batch, 'failed_at', None),
            "output_file_id": getattr(batch, 'output_file_id', None),
            "error_file_id": getattr(batch, 'error_file_id', None),
            "request_counts": getattr(batch, 'request_counts', None)
        }

        return result

    except Exception as e:
        print(f"Error checking batch status: {e}")
        return None

def file_check(status):
  file_response=None
  if status["output_file_id"] is not None:
    print("Saving file response")
    file_response = client.files.content(status["output_file_id"])
  else:
    print("File response not available")
  return file_response

In [ ]:
batch_id = create_batch_job(client, "batch_requests_fully_clean1.jsonl", "create full clean first")

In [ ]:
status = check_batch_status(client, "batch_6895610dd0708190866774579d0f3f7e")

Batch status: Batch(id='batch_6895610dd0708190866774579d0f3f7e', completion_window='24h', created_at=1754620173, endpoint='/v1/chat/completions', input_file_id='file-6LoNtorYAUZWzmuTextM1s', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1754624078, error_file_id=None, errors=None, expired_at=None, expires_at=1754706573, failed_at=None, finalizing_at=1754622275, in_progress_at=1754620180, metadata={'description': 'create full clean first'}, output_file_id='file-37v8Pv4fVJi8Wwy89VyyAn', request_counts=BatchRequestCounts(completed=17200, failed=0, total=17200))


In [ ]:
file_response1 = file_check(status)

Saving file response


In [ ]:
print(file_response1.text)

In [ ]:
batch_id2 = create_batch_job(client, "batch_requests_fully_clean2.jsonl", "create full clean second")

In [ ]:
status2 = check_batch_status(client, "batch_6895612a7358819095cbac9c64db869c")

Batch status: Batch(id='batch_6895612a7358819095cbac9c64db869c', completion_window='24h', created_at=1754620202, endpoint='/v1/chat/completions', input_file_id='file-3YTUmt5MP8FMespRgVtGVX', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1754633902, error_file_id=None, errors=None, expired_at=None, expires_at=1754706602, failed_at=None, finalizing_at=1754632706, in_progress_at=1754620211, metadata={'description': 'create full clean second'}, output_file_id='file-JcpLKbhdHRbWe92CQbuGue', request_counts=BatchRequestCounts(completed=17200, failed=0, total=17200))


In [ ]:
file_response2 = file_check(status2)

Saving file response


In [ ]:
batch_id3 = create_batch_job(client, "batch_requests_fully_clean3.jsonl", "create full clean third")

In [ ]:
status3 = check_batch_status(client, "batch_68956147b1788190a26a1c79348a11cc")

Batch status: Batch(id='batch_68956147b1788190a26a1c79348a11cc', completion_window='24h', created_at=1754620231, endpoint='/v1/chat/completions', input_file_id='file-EzBFqr3W5dCynTSt54k8Nq', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1754634086, error_file_id=None, errors=None, expired_at=None, expires_at=1754706631, failed_at=None, finalizing_at=1754632614, in_progress_at=1754620241, metadata={'description': 'create full clean third'}, output_file_id='file-48xdmu3BYtZj2n6f2ExS2Q', request_counts=BatchRequestCounts(completed=17200, failed=0, total=17200))


In [ ]:
file_response3 = file_check(status3)

Saving file response


In [ ]:
batch_id4 = create_batch_job(client, "batch_requests_fully_clean4.jsonl", "create full clean third")

In [ ]:
status4 = check_batch_status(client, "batch_689561537814819091347f223fa7e24b")

Batch status: Batch(id='batch_689561537814819091347f223fa7e24b', completion_window='24h', created_at=1754620243, endpoint='/v1/chat/completions', input_file_id='file-7k9HVQR1PHZDu1Fc3To6w6', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1754623623, error_file_id=None, errors=None, expired_at=None, expires_at=1754706643, failed_at=None, finalizing_at=1754622321, in_progress_at=1754620251, metadata={'description': 'create full clean third'}, output_file_id='file-DCVGSGBUXRDMuYPAcsePoG', request_counts=BatchRequestCounts(completed=17200, failed=0, total=17200))


In [ ]:
file_response4 = file_check(status4)

Saving file response


In [ ]:
batch_id5 = create_batch_job(client, "batch_requests_fully_clean5.jsonl", "create full clean fifth")

In [ ]:
status5 = check_batch_status(client, "batch_68956160084c81908925424a3d71b018")

Batch status: Batch(id='batch_68956160084c81908925424a3d71b018', completion_window='24h', created_at=1754620256, endpoint='/v1/chat/completions', input_file_id='file-YH7qcdAY4Xer2MRsmNNHfT', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1754623657, error_file_id=None, errors=None, expired_at=None, expires_at=1754706656, failed_at=None, finalizing_at=1754621859, in_progress_at=1754620261, metadata={'description': 'create full clean fifth'}, output_file_id='file-7SC38a7E1cstFD7LyTCUnP', request_counts=BatchRequestCounts(completed=17200, failed=0, total=17200))


In [ ]:
file_response5 = file_check(status5)

Saving file response


In [ ]:
import json
from datasets import Dataset, DatasetDict
from typing import Optional
import re

def parse_jsonl_to_dataset(jsonl_content: str, split_name: str = "train") -> DatasetDict:

    data_records = []

    lines = jsonl_content.strip().split('\n')
    for line_num, line in enumerate(lines):
        if not line.strip():
            continue
        try:
            obj = json.loads(line)
            try:
                record = {}

                # Extract basic identifiers
                record["id"] = obj.get("id", f"unknown_{line_num}")
                record["custom_id"] = obj.get("custom_id", "")

                # Skip if there's an error
                if obj.get("error"):
                    continue

                # Extract response data
                response = obj.get("response", {})
                if response.get("status_code") != 200:
                    continue

                body = response.get("body", {})
                choices = body.get("choices", [])

                if choices:
                    # Extract assistant response content
                    message = choices[0].get("message", {})
                    content = message.get("content", "")

                    original_email, rewritten_email = extract_email_pair(content)

                    # Only store records where both emails were successfully extracted
                    if original_email and rewritten_email:
                        record["original_email"] = original_email
                        record["rewritten_email"] = rewritten_email
                        record["model"] = body.get("model", "")
                        record["finish_reason"] = choices[0].get("finish_reason", "")
                        data_records.append(record)

            except Exception:
                continue
        except:
          continue
    dataset = Dataset.from_list(data_records)
    return DatasetDict({split_name: dataset})


def extract_email_pair(content: str) -> tuple[Optional[str], Optional[str]]:
    """
    Extract original and rewritten email content, ensuring no numbered prefixes are included.
    """
    try:
        # Primary pattern: Look for numbered sections with headers
        pattern1 = r"1\.\s*(?:Email Content|Original Email|Original):\s*\n(.*?)(?=2\.\s*(?:Rewritten email|Rewritten Email|Rewritten):|$)"
        pattern2 = r"2\.\s*(?:Rewritten email|Rewritten Email|Rewritten):\s*\n(.*?)$"

        match1 = re.search(pattern1, content, re.DOTALL | re.IGNORECASE)
        match2 = re.search(pattern2, content, re.DOTALL | re.IGNORECASE)

        if match1 and match2:
            original_email = clean_email_content(match1.group(1))
            rewritten_email = clean_email_content(match2.group(1))
            return original_email, rewritten_email

        # Fallback: Look for Subject: patterns
        subjects = re.findall(r'Subject:.*?(?=Subject:|$)', content, re.DOTALL | re.IGNORECASE)
        if len(subjects) >= 2:
            original_email = clean_email_content(subjects[0])
            rewritten_email = clean_email_content(subjects[1])
            return original_email, rewritten_email

        # Another fallback: Split by double newlines followed by Subject:
        sections = re.split(r'\n\s*\n(?=Subject:)', content, flags=re.IGNORECASE)
        if len(sections) >= 2:
            email_sections = [s.strip() for s in sections if 'subject:' in s.lower()]
            if len(email_sections) >= 2:
                original_email = clean_email_content(email_sections[0])
                rewritten_email = clean_email_content(email_sections[1])
                return original_email, rewritten_email

        return None, None

    except Exception as e:
        print(f"Error extracting email pair: {e}")
        return None, None


def clean_email_content(email_text: str) -> str:
    """
    Clean email content by removing numbered prefixes that appear at the start of the content.
    """
    if not email_text:
        return email_text

    cleaned = email_text.strip()

    # Remove section headers like "1. Email Content:" or "2. Rewritten email:" at the start
    cleaned = re.sub(r'^\s*[12]\.\s*(?:Email Content|Original Email|Original|Rewritten email|Rewritten Email|Rewritten):\s*\n?', '', cleaned, flags=re.IGNORECASE)

    # Remove any numbered prefix at the very start of the content (like "1. Subject:" -> "Subject:")
    cleaned = re.sub(r'^\s*\d+\.\s+', '', cleaned)

    # Clean up extra whitespace
    cleaned = re.sub(r'\n\s*\n\s*\n+', '\n\n', cleaned)  # Multiple blank lines to double
    cleaned = cleaned.strip()

    return cleaned


def save_dataset(dataset_dict: DatasetDict, output_path: str):
    dataset_dict.save_to_disk(output_path)

In [ ]:
dataset_dict1 = parse_jsonl_to_dataset(file_response1.text)

In [ ]:
dataset_dict2 = parse_jsonl_to_dataset(file_response2.text)

In [ ]:
dataset_dict3 = parse_jsonl_to_dataset(file_response3.text)

In [ ]:
dataset_dict4 = parse_jsonl_to_dataset(file_response4.text)

In [ ]:
dataset_dict5 = parse_jsonl_to_dataset(file_response5.text)

In [ ]:
from datasets import concatenate_datasets, DatasetDict

dict1 = concatenate_datasets([dataset_dict1["train"], dataset_dict2["train"],dataset_dict3["train"],dataset_dict4["train"], dataset_dict5["train"]])
full_dataset_dict = DatasetDict({"train":dict1})

In [ ]:
full_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'custom_id', 'original_email', 'rewritten_email', 'model', 'finish_reason'],
        num_rows: 86000
    })
})

In [ ]:
from datasets import DatasetDict, Dataset

def transform_dataset(full_dataset_dict):

    def process_split(dataset):
        # Create new dataset with transformed structure
        new_data = {
            'id': dataset['id'],
            'custom_id': dataset['custom_id'],
            'body': dataset['original_email'],  # Rename original_email to body
            'pair': dataset['rewritten_email'],  # Rename rewritten_email to pair
            'category': ['clean'] * len(dataset)  # Add 'clean' category to every element
        }

        return Dataset.from_dict(new_data)

    # Transform each split in the dataset
    transformed_dict = {}
    for split_name, dataset in full_dataset_dict.items():
        transformed_dict[split_name] = process_split(dataset)

    return DatasetDict(transformed_dict)


full_dataset_dict = transform_dataset(full_dataset_dict)

In [ ]:
full_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'custom_id', 'body', 'pair', 'category'],
        num_rows: 86000
    })
})

# Mount Data to Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


# Tokenization and Embedding Generation

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import logging
import os
from transformers import TrainerCallback
import numpy as np
from datasets import load_dataset, Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
    InputExample,
)
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import BinaryClassificationEvaluator
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import EarlyStoppingCallback

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using: ", device)
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

# Prepare datasets in the format expected by SentenceTransformerTrainer
def prepare_dataset_for_trainer(dataset):
    """Prepare dataset in the format expected by SentenceTransformerTrainer"""
    # The trainer expects a dataset with 'sentence1', 'sentence2', and 'label' columns
    prepared_data = []
    for item in dataset:
        prepared_data.append({
            'sentence1': item['body'],
            'sentence2': item['pair'],
            'label': float(item['similarity'])
        })
    return Dataset.from_list(prepared_data)

# Convert dataset to InputExample format for evaluator
def convert_to_input_examples(dataset):
    """Convert dataset to InputExample format required for evaluation"""
    examples = []
    for item in dataset:
        examples.append(InputExample(
            texts=[item['body'], item['pair']],
            label=float(item['similarity'])
        ))
    return examples

# Train on FULL dataset and evaluate on FULL test dataset
print("Dataset sizes:")
# print(f"Training: {len(train_data['train'])} (using FULL dataset)")
print(f"Training: {len(train_dataset['train'])} (using FULL dataset)")
# print(f"Test: {len(test_data['train'])} (using FULL dataset)")
print(f"Test: {len(test_dataset['train'])} (using FULL dataset)")
# print(f"Using full training dataset ({len(train_data['train'])} samples)")
print(f"Using full training dataset ({len(train_dataset['train'])} samples)")
# print(f"Using full test dataset ({len(test_data['train'])} samples)")
print(f"Using full test dataset ({len(test_data['train'])} samples)")

# Prepare datasets for training
print("Preparing FULL training dataset...")
train_dataset_prepared = prepare_dataset_for_trainer(train_dataset["train"])

print("Preparing FULL test dataset...")
test_dataset_prepared = prepare_dataset_for_trainer(test_dataset["train"])

# Convert datasets for evaluation
print("Converting FULL test dataset to InputExample format for evaluation...")
test_examples = convert_to_input_examples(test_dataset["train"])

class EarlyStoppingOnTrainLossCallback(TrainerCallback):
    def __init__(self):
        self.best_loss = np.inf
        self.num_bad_steps = 0
    def on_log(self, args, state, control, logs=None, **kwargs):
        # logs contain training info, e.g. loss
        if logs is None:
            return
        current_loss = logs.get("loss")
        if current_loss is None:
            return
        if current_loss <= 0.003:
            control.should_training_stop=True
class ModelSavingCallback(TrainerCallback):
    """Custom callback to save model at regular intervals"""
    def __init__(self, save_every_n_steps=500):
        self.save_every_n_steps = save_every_n_steps
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.save_every_n_steps == 0:
            checkpoint_dir = f"{args.output_dir}/checkpoint-{state.global_step}"
            os.makedirs(checkpoint_dir, exist_ok=True)
            kwargs['model'].save(checkpoint_dir)
            print(f"Model saved at step {state.global_step} to {checkpoint_dir}")
logging.basicConfig(level=logging.INFO)
# Initialize loss function
loss = CosineSimilarityLoss(model)
# Prepare evaluator with your test dataset (you already have this)
test_evaluator = BinaryClassificationEvaluator(
    sentences1=[example.texts[0] for example in test_examples],
    sentences2=[example.texts[1] for example in test_examples],
    labels=[int(example.label) for example in test_examples],
    similarity_fn_names=["cosine"],
    show_progress_bar=True,
)
# Create output directory if it doesn't exist
output_dir = "models/mpnet-base-all-nli-triplet"
os.makedirs(output_dir, exist_ok=True)
# Update training args with proper saving configuration
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    # Enable evaluation and saving during training
    eval_strategy="no",              # Evaluate every N steps
    save_strategy="steps",              # Save every N steps
    save_steps=500,                     # Save every 500 steps
    save_total_limit=3,                 # Keep only 3 most recent checkpoints
    logging_steps=100,
    metric_for_best_model="eval_cosine_f1",  # Use evaluation F1 score
    greater_is_better=True,
    report_to="none",
)
# Create trainer with callbacks
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset_prepared,
    eval_dataset=None,  # You can add validation dataset here if available
    loss=loss,
    evaluator=test_evaluator,  # This will be used for evaluation during training
    callbacks=[
        EarlyStoppingOnTrainLossCallback(),
        ModelSavingCallback(save_every_n_steps=500)  # Additional custom saving
    ]
)
# Train the model
print("Starting training...")
print(f"Training on {len(train_dataset_prepared)} samples...")
print(f"Model checkpoints will be saved to: {output_dir}")
trainer.train()
# The best model is automatically loaded at the end due to load_best_model_at_end=True
print("Training completed. Best model has been loaded automatically.")
# Save the final trained model
final_model_path = "models/mpnet-base-all-nli-triplet-final"
print(f"Saving final trained model to {final_model_path}...")
model.save(final_model_path)
print("Final model saved!")
# Evaluate on test dataset
print("\n" + "="*50)
print("TRAINING COMPLETED - EVALUATING ON TEST DATA")
print("="*50)
print("Evaluating trained model on test dataset...")
final_eval_results = test_evaluator(model)
print(f"Final test evaluation results: {final_eval_results}")
# List all saved checkpoints
print("\nSaved checkpoints:")
if os.path.exists(output_dir):
    checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
    checkpoints.sort(key=lambda x: int(x.split("-")[1]))  # Sort by step number
    for checkpoint in checkpoints:
        print(f"  - {os.path.join(output_dir, checkpoint)}")
else:
    print("  No checkpoints found")

In [ ]:
batches = split_dataset(dataset_clean_with_pairs, number=16)
batches

In [ ]:
len(batches)

In [ ]:
import json
import os
import torch
import numpy as np
from google.colab import drive
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from functools import partial
import gc

def process_batch_on_gpu(batch_data, features, device):
    """Process a batch of data on GPU for faster operations"""
    try:
        batch_rows = []

        # Move batch to GPU if it's not already there
        if isinstance(batch_data, dict):
            gpu_batch = {}
            for feature in features:
                if feature in batch_data:
                    data = batch_data[feature]
                    if torch.is_tensor(data):
                        gpu_batch[feature] = data.to(device) if data.device != device else data
                    else:
                        # Convert to tensor and move to GPU if numeric
                        try:
                            if isinstance(data, (list, np.ndarray)):
                                gpu_batch[feature] = torch.tensor(data, device=device)
                            else:
                                gpu_batch[feature] = data
                        except:
                            gpu_batch[feature] = data
                else:
                    gpu_batch[feature] = None

            # Convert back to CPU for JSON serialization
            cpu_row = {}
            for feature in features:
                if torch.is_tensor(gpu_batch[feature]):
                    cpu_row[feature] = gpu_batch[feature].cpu().tolist()
                elif isinstance(gpu_batch[feature], np.ndarray):
                    cpu_row[feature] = gpu_batch[feature].tolist()
                else:
                    cpu_row[feature] = gpu_batch[feature]

            batch_rows.append(cpu_row)

        return batch_rows

    except Exception as e:
        print(f"GPU processing failed, falling back to CPU: {e}")
        # Fallback to CPU processing
        batch_rows = []
        for feature in features:
            row = {}
            for feat in features:
                if isinstance(batch_data, dict) and feat in batch_data:
                    data = batch_data[feat]
                    if torch.is_tensor(data):
                        row[feat] = data.cpu().tolist()
                    elif isinstance(data, np.ndarray):
                        row[feat] = data.tolist()
                    else:
                        row[feat] = data
                else:
                    row[feat] = None
            batch_rows.append(row)
        return batch_rows

def process_dataset_parallel(dataset, phase_name, batch_size=1000, use_gpu=True, num_workers=None):
    """Process dataset in parallel batches with optional GPU acceleration"""

    # Setup device
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"Using GPU: {torch.cuda.get_device_name()}")
    else:
        device = torch.device('cpu')
        print("Using CPU for processing")

    # Set number of workers
    if num_workers is None:
        num_workers = min(4, mp.cpu_count())  # Conservative for memory

    features = list(dataset.features.keys())
    total_rows = len(dataset)

    print(f"Processing {phase_name} with {num_workers} workers in batches of {batch_size}...")

    all_data = []

    # Process in batches
    for start_idx in tqdm(range(0, total_rows, batch_size), desc=f"Processing {phase_name}"):
        end_idx = min(start_idx + batch_size, total_rows)

        # Get batch data
        batch_indices = list(range(start_idx, end_idx))
        batch_data = dataset.select(batch_indices)

        # Convert batch to format suitable for GPU processing
        if use_gpu and torch.cuda.is_available():
            try:
                # Try to use GPU format if available
                batch_data.set_format(type='torch', device=device)
            except:
                pass

        # Process batch (could be parallelized further if needed)
        batch_rows = []
        for i in range(len(batch_data)):
            row = {}
            for feature in features:
                data = batch_data[i][feature]
                if torch.is_tensor(data):
                    row[feature] = data.cpu().tolist() if data.is_cuda else data.tolist()
                elif isinstance(data, np.ndarray):
                    row[feature] = data.tolist()
                else:
                    row[feature] = data
            batch_rows.append(row)

        all_data.extend(batch_rows)

        # Clear GPU cache periodically
        if use_gpu and torch.cuda.is_available():
            torch.cuda.empty_cache()

    return {
        'features': features,
        'num_rows': len(all_data),
        'data': all_data
    }

def save_data(dataset_dict, filename='dataset.json', output_dir='/content/drive/MyDrive/Algoverse/',
              mount_drive=True, use_gpu=True, batch_size=1000, num_workers=None, use_parallel_phases=True):
    """
    Save dataset with GPU acceleration and parallel processing

    Args:
        dataset_dict: Dictionary of datasets to save
        filename: Output filename
        output_dir: Output directory
        mount_drive: Whether to mount Google Drive
        use_gpu: Whether to use GPU acceleration
        batch_size: Batch size for processing
        num_workers: Number of parallel workers
        use_parallel_phases: Whether to process phases in parallel
    """

    if mount_drive:
        drive.mount('/content/drive')

    os.makedirs(output_dir, exist_ok=True)

    # Check GPU availability
    if use_gpu and torch.cuda.is_available():
        print(f"GPU acceleration enabled: {torch.cuda.get_device_name()}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        # Clear cache at start
        torch.cuda.empty_cache()
    else:
        print("Using CPU processing")
        use_gpu = False

    # Set number of workers
    if num_workers is None:
        num_workers = min(4, mp.cpu_count())

    json_data = {}

    if use_parallel_phases and len(dataset_dict) > 1:
        # Process phases in parallel
        print(f"Processing {len(dataset_dict)} phases in parallel...")

        with ThreadPoolExecutor(max_workers=min(len(dataset_dict), num_workers)) as executor:
            # Submit all phase processing tasks
            future_to_phase = {
                executor.submit(process_dataset_parallel, dataset, phase_name, batch_size, use_gpu, 1): phase_name
                for phase_name, dataset in dataset_dict.items()
            }

            # Collect results
            for future in tqdm(future_to_phase, desc="Processing phases"):
                phase_name = future_to_phase[future]
                try:
                    result = future.result()
                    json_data[phase_name] = result
                    print(f"Completed {phase_name}: {result['num_rows']} rows")
                except Exception as e:
                    print(f"Error processing {phase_name}: {e}")
    else:
        # Process phases sequentially
        for phase_name, dataset in dataset_dict.items():
            result = process_dataset_parallel(dataset, phase_name, batch_size, use_gpu, num_workers)
            json_data[phase_name] = result
            print(f"Completed {phase_name}: {result['num_rows']} rows")

    # Save to file
    output_path = os.path.join(output_dir, filename)
    print(f"Saving data to {output_path}...")

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)

        print(f"Successfully saved dataset to {output_path}")
        print(f"File size: {os.path.getsize(output_path) / 1e6:.1f} MB")

    except Exception as e:
        print(f"Error saving file: {e}")
        # Try saving without indentation to save space
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, default=str)
        print(f"Saved without formatting due to memory constraints")

    # Clean up GPU memory
    if use_gpu and torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

    return output_path


In [ ]:
import json
from datasets import Dataset, DatasetDict
import os
from google.colab import drive
from tqdm import tqdm
def load_data(json_file_path, mount_drive=True):
  if mount_drive:
        drive.mount('/content/drive')
  print(f"Loading data from {json_file_path}...")
  with open(json_file_path, 'r', encoding='utf-8') as f:
      json_data = json.load(f)

  dataset_dict = {}
  for phase_name, phase_info in json_data.items():
      print(f"Processing {phase_name}...")

      dataset_dict[phase_name] = Dataset.from_list(phase_info['data'])

  tokenized_dataset_dict = DatasetDict(dataset_dict)

  print("Successfully loaded DatasetDict!")

  return tokenized_dataset_dict

In [ ]:
full_dataset_dict["train"]=full_dataset_dict["train"].add_column("similarity", [1.0] * len(full_dataset_dict["train"]))

In [ ]:
full_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'custom_id', 'body', 'pair', 'category', 'similarity'],
        num_rows: 86000
    })
})

In [ ]:
from datasets import concatenate_datasets, DatasetDict
import torch
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

def load_and_combine_datasets(use_gpu=True, num_workers=None):
    """Load datasets from token_0 to token_31 and combine them into one large dataset

    Args:
        use_gpu (bool): Whether to use GPU for operations when possible
        num_workers (int): Number of parallel workers for loading (None = auto-detect)
    """

    # Check GPU availability
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"GPU detected: {torch.cuda.get_device_name()}")
        print(f"GPU memory available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    else:
        device = torch.device('cpu')
        print("Using CPU for operations")

    # Set number of workers for parallel loading
    if num_workers is None:
        num_workers = min(8, mp.cpu_count())  # Reasonable default

    all_datasets = []
    base_path = "/content/drive/MyDrive/Algoverse/token_"

    print(f"Loading datasets with {num_workers} parallel workers...")

    def load_single_dataset(i):
        """Helper function to load a single dataset"""
        dataset_path = f"{base_path}{i}"
        try:
            dataset = load_data(dataset_path)
            return i, dataset
        except Exception as e:
            print(f"Warning: Could not load dataset {i}: {e}")
            return i, None

    # Use ThreadPoolExecutor for parallel loading
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all loading tasks
        futures = [executor.submit(load_single_dataset, i) for i in range(32)]

        # Collect results as they complete
        for future in futures:
            i, dataset = future.result()
            if dataset is not None:
                print(f"Loaded dataset {i}")

                # Handle different dataset structures
                if isinstance(dataset, DatasetDict):
                    for split_name, split_data in dataset.items():
                        print(f"  Split '{split_name}': {len(split_data)} rows")
                        # Set format for GPU if available and data supports it
                        if use_gpu and torch.cuda.is_available():
                            try:
                                split_data.set_format(type='torch', device=device)
                            except:
                                pass  # Some datasets might not support torch format
                        all_datasets.append(split_data)
                else:
                    print(f"  Dataset: {len(dataset)} rows")
                    # Set format for GPU if available
                    if use_gpu and torch.cuda.is_available():
                        try:
                            dataset.set_format(type='torch', device=device)
                        except:
                            pass
                    all_datasets.append(dataset)

    if not all_datasets:
        raise ValueError("No datasets were successfully loaded!")

    # Combine all datasets (this operation is CPU-bound)
    print(f"\nCombining {len(all_datasets)} dataset splits...")
    combined_dataset = concatenate_datasets(all_datasets)

    # Set format for GPU on final dataset if requested
    if use_gpu and torch.cuda.is_available():
        try:
            combined_dataset.set_format(type='torch', device=device)
            print(f"Dataset moved to GPU: {device}")
        except Exception as e:
            print(f"Could not move dataset to GPU: {e}")

    # Create a DatasetDict with train split
    final_dataset = DatasetDict({
        'train': combined_dataset
    })

    print(f"Successfully combined all datasets!")
    print(f"Final dataset shape: {len(combined_dataset)} rows")
    print(f"Features: {list(combined_dataset.features.keys())}")

    return final_dataset

combined_dataset = load_and_combine_datasets(use_gpu=True, num_workers=8)

print(f"\nFinal combined dataset:")
print(combined_dataset)

GPU detected: NVIDIA A100-SXM4-40GB
GPU memory available: 42.5 GB
Loading datasets with 8 parallel workers...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from /content/drive/MyDrive/Algoverse/token_0...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from /content/drive/MyDrive/Algoverse/token_1...
Processing Phase1...
Processing Phase1...
Successfully loaded DatasetDict!
Successfully loaded DatasetDict!
Loaded dataset 0
  Split 'Phase1': 8610 rows
Loaded dataset 1
  Split 'Phase1': 8610 rows
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from /content/drive/MyDrive/Algoverse/token_3...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remou

In [ ]:
combined_dataset["train"] = combined_dataset["train"].add_column("similarity", [0.0] * len(combined_dataset["train"]))

In [ ]:
combined_dataset["train"] = combined_dataset["train"].remove_columns(["body_embeddings", "pair_embeddings"])

In [ ]:
from datasets import Dataset, DatasetDict

def filter_dataset_columns(dataset, keep_columns=None):
    """
    Filter dataset to keep only specified columns.
    Works with both Dataset and DatasetDict objects.

    Args:
        dataset: The input dataset (Dataset or DatasetDict)
        keep_columns: List of column names to keep (default: ['body', 'pair', 'body_embeddings', 'pair_embeddings'])

    Returns:
        New filtered dataset with only the specified columns (same type as input)
    """

    if keep_columns is None:
        keep_columns = ['body', 'pair', 'body_embeddings', 'pair_embeddings']

    # Handle DatasetDict
    if isinstance(dataset, DatasetDict):
        filtered_dict = {}

        for split_name, split_dataset in dataset.items():
            print(f"\nProcessing split: {split_name}")

            # Check which columns exist in this split
            available_columns = split_dataset.column_names
            columns_to_keep = [col for col in keep_columns if col in available_columns]

            print(f"Original columns: {available_columns}")
            print(f"Requested columns: {keep_columns}")
            print(f"Columns to keep (available): {columns_to_keep}")

            missing_columns = [col for col in keep_columns if col not in available_columns]
            if missing_columns:
                print(f"Warning: These requested columns don't exist: {missing_columns}")

            # Filter this split
            filtered_dict[split_name] = split_dataset.select_columns(columns_to_keep)

        return DatasetDict(filtered_dict)

    # Handle single Dataset
    else:
        # Check which columns exist in the dataset
        available_columns = dataset.column_names
        columns_to_keep = [col for col in keep_columns if col in available_columns]

        # Print info about what we're keeping vs what's missing
        print(f"Original columns: {available_columns}")
        print(f"Requested columns: {keep_columns}")
        print(f"Columns to keep (available): {columns_to_keep}")

        missing_columns = [col for col in keep_columns if col not in available_columns]
        if missing_columns:
            print(f"Warning: These requested columns don't exist: {missing_columns}")

        # Create and return a new filtered dataset
        filtered_dataset = dataset.select_columns(columns_to_keep)

        return filtered_dataset

# Example usage:
def process_dataset(input_dataset):
    """
    Process a dataset by filtering to keep only specific columns.
    Works with both Dataset and DatasetDict objects.

    Args:
        input_dataset: The dataset to process (Dataset or DatasetDict)

    Returns:
        New filtered dataset (same type as input)
    """
    return filter_dataset_columns(
        input_dataset,
        keep_columns=['body', 'pair', "category", "similarity"]
    )

In [ ]:
filtered_dataset = process_dataset(combined_dataset)
filtered_dataset

In [ ]:
filtered_clean = process_dataset(full_dataset_dict)
filtered_clean

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

def sample_dataset_stratified(dataset_dict, sample_fraction=0.5, category_column='category', random_state=42):
    np.random.seed(random_state)
    sampled_dict = {}
    remaining_dict = {}

    for split_name, dataset in dataset_dict.items():
        df = dataset.to_pandas()
        original_counts = df[category_column].value_counts()
        target_size = int(len(df) * sample_fraction)

        print(f"{split_name}: {len(df)} → {target_size} rows sampled, {len(df) - target_size} rows remaining")

        sampled_dfs = []
        remaining_dfs = []

        for category in original_counts.index:
            category_df = df[df[category_column] == category]
            category_sample_size = max(1, int(len(category_df) * sample_fraction))

            sampled_category = category_df.sample(
                n=min(category_sample_size, len(category_df)),
                random_state=random_state
            )

            # Get remaining data by excluding sampled indices
            remaining_category = category_df.drop(sampled_category.index)

            sampled_dfs.append(sampled_category)
            remaining_dfs.append(remaining_category)

        # Combine and shuffle sampled data
        sampled_df = pd.concat(sampled_dfs, ignore_index=True)
        sampled_df = sampled_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
        sampled_dict[split_name] = Dataset.from_pandas(sampled_df)

        # Combine and shuffle remaining data
        remaining_df = pd.concat(remaining_dfs, ignore_index=True)
        remaining_df = remaining_df.sample(frac=1, random_state=random_state + 1).reset_index(drop=True)
        remaining_dict[split_name] = Dataset.from_pandas(remaining_df)

    return DatasetDict(sampled_dict), DatasetDict(remaining_dict)

def verify_distribution(original_dict, sampled_dict, remaining_dict=None, category_column='category'):
    print("\nDistribution Verification:")
    for split_name in original_dict.keys():
        original_df = original_dict[split_name].to_pandas()
        sampled_df = sampled_dict[split_name].to_pandas()

        original_dist = original_df[category_column].value_counts(normalize=True)
        sampled_dist = sampled_df[category_column].value_counts(normalize=True)

        print(f"\n{split_name}:")
        print(f"  Original: {len(original_df)} rows")
        print(f"  Sampled:  {len(sampled_df)} rows")

        if remaining_dict:
            remaining_df = remaining_dict[split_name].to_pandas()
            remaining_dist = remaining_df[category_column].value_counts(normalize=True)
            print(f"  Remaining: {len(remaining_df)} rows")

        print("  Category distributions:")
        for category in original_dist.index:
            orig_pct = original_dist[category] * 100
            samp_pct = sampled_dist.get(category, 0) * 100
            print(f"    {category}: Original {orig_pct:.1f}% → Sampled {samp_pct:.1f}%", end="")

            if remaining_dict:
                rem_pct = remaining_dist.get(category, 0) * 100
                print(f" | Remaining {rem_pct:.1f}%")
            else:
                print()

sampled_dataset, remaining_dataset = sample_dataset_stratified(filtered_dataset, sample_fraction=0.5)
verify_distribution(filtered_dataset, sampled_dataset, remaining_dataset)

In [ ]:
s1, s2 = sample_dataset_stratified(sampled_dataset, sample_fraction=0.7)
t1, t2 = sample_dataset_stratified(filtered_clean, sample_fraction=0.7)

In [ ]:
from datasets import concatenate_datasets, DatasetDict
train_data = DatasetDict({"train":concatenate_datasets([s1["train"], t1["train"]])})
test_data = DatasetDict({"train":concatenate_datasets([s2["train"], t2["train"]])})

# M

In [ ]:
import logging
import os
from transformers import TrainerCallback
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
    InputExample,
)
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import BinaryClassificationEvaluator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import EarlyStoppingCallback

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Initialize Llama 3 model for sentence transformers
# Note: Using a Llama 3-based sentence transformer model
print("Loading Llama 3 sentence transformer model...")
try:
    # Try McGill-NLP's Llama 3 sentence transformer first
    model = SentenceTransformer('McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp-supervised',
                               device=device, trust_remote_code=True)
    print("Successfully loaded McGill-NLP Llama 3 model")
except Exception as e:
    print(f"Failed to load McGill-NLP model: {e}")
    try:
        # Fallback to another Llama 3 sentence transformer
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
        print("Loaded fallback model (consider using a proper Llama 3 sentence transformer)")
    except Exception as e2:
        print(f"Fallback also failed: {e2}")
        raise

class EarlyStoppingOnTrainLossCallback(TrainerCallback):
    def __init__(self):
        self.best_loss = np.inf
        self.num_bad_steps = 0

    def on_log(self, args, state, control, logs=None, **kwargs):
        # logs contain training info, e.g. loss
        if logs is None:
            return
        current_loss = logs.get("loss")
        if current_loss is None:
            return
        if current_loss <= 0.003:
            control.should_training_stop = True

class ModelSavingCallback(TrainerCallback):
    """Custom callback to save model at regular intervals"""
    def __init__(self, save_every_n_steps=500):
        self.save_every_n_steps = save_every_n_steps

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.save_every_n_steps == 0:
            checkpoint_dir = f"{args.output_dir}/checkpoint-{state.global_step}"
            os.makedirs(checkpoint_dir, exist_ok=True)
            kwargs['model'].save(checkpoint_dir)
            print(f"Model saved at step {state.global_step} to {checkpoint_dir}")

# Prepare datasets in the format expected by SentenceTransformerTrainer
def prepare_dataset_for_trainer(dataset):
    """Prepare dataset in the format expected by SentenceTransformerTrainer"""
    # The trainer expects a dataset with 'sentence1', 'sentence2', and 'label' columns
    prepared_data = []
    for item in dataset:
        prepared_data.append({
            'sentence1': item['body'],
            'sentence2': item['pair'],
            'label': float(item['similarity'])
        })
    return Dataset.from_list(prepared_data)

# Convert dataset to InputExample format for evaluator
def convert_to_input_examples(dataset):
    """Convert dataset to InputExample format required for evaluation"""
    examples = []
    for item in dataset:
        examples.append(InputExample(
            texts=[item['body'], item['pair']],
            label=float(item['similarity'])
        ))
    return examples

# Load your datasets (assuming they're already loaded as train_dataset and test_dataset)
# You'll need to uncomment and modify these lines based on your actual dataset loading
# train_dataset = load_dataset("your_dataset_name", split="train")
# test_dataset = load_dataset("your_dataset_name", split="test")

print("Dataset sizes:")
print(f"Training: {len(train_dataset['train'])} (using FULL dataset)")
print(f"Test: {len(test_dataset['train'])} (using FULL dataset)")

# Prepare datasets for training
print("Preparing FULL training dataset...")
train_dataset_prepared = prepare_dataset_for_trainer(train_dataset["train"])

print("Preparing FULL test dataset...")
test_dataset_prepared = prepare_dataset_for_trainer(test_dataset["train"])

# Convert datasets for evaluation
print("Converting FULL test dataset to InputExample format for evaluation...")
test_examples = convert_to_input_examples(test_dataset["train"])

# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize loss function
loss = CosineSimilarityLoss(model)

# Prepare evaluator with your test dataset
test_evaluator = BinaryClassificationEvaluator(
    sentences1=[example.texts[0] for example in test_examples],
    sentences2=[example.texts[1] for example in test_examples],
    labels=[int(example.label) for example in test_examples],
    similarity_fn_names=["cosine"],
    show_progress_bar=True,
)

# Create output directory if it doesn't exist
output_dir = "models/llama3-sentence-transformer-trained"
os.makedirs(output_dir, exist_ok=True)

# Update training args with proper saving configuration
# Note: Reduced batch size and learning rate for Llama 3 which is larger
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=8,  # Reduced for larger Llama 3 model
    gradient_accumulation_steps=4,   # Increased to maintain effective batch size
    learning_rate=1e-5,             # Reduced learning rate for large model
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    # Enable evaluation and saving during training
    eval_strategy="no",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    logging_steps=50,               # More frequent logging for monitoring
    metric_for_best_model="eval_cosine_f1",
    greater_is_better=True,
    report_to="none",
    dataloader_num_workers=0,       # Set to 0 to avoid multiprocessing issues
)

# Create trainer with callbacks
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset_prepared,
    eval_dataset=None,
    loss=loss,
    evaluator=test_evaluator,
    callbacks=[
        EarlyStoppingOnTrainLossCallback(),
        ModelSavingCallback(save_every_n_steps=500)
    ]
)

# Train the model
print("Starting training with Llama 3...")
print(f"Training on {len(train_dataset_prepared)} samples...")
print(f"Model checkpoints will be saved to: {output_dir}")
print(f"Using model: {model}")

trainer.train()

print("Training completed. Best model has been loaded automatically.")

# Save the final trained model
final_model_path = "models/llama3-sentence-transformer-final"
print(f"Saving final trained model to {final_model_path}...")
model.save(final_model_path)
print("Final model saved!")

# Evaluate on test dataset
print("\n" + "="*50)
print("TRAINING COMPLETED - EVALUATING ON TEST DATA")
print("="*50)
print("Evaluating trained Llama 3 model on test dataset...")
final_eval_results = test_evaluator(model)
print(f"Final test evaluation results: {final_eval_results}")

# List all saved checkpoints
print("\nSaved checkpoints:")
if os.path.exists(output_dir):
    checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
    checkpoints.sort(key=lambda x: int(x.split("-")[1]))
    for checkpoint in checkpoints:
        print(f"  - {os.path.join(output_dir, checkpoint)}")
else:
    print("  No checkpoints found")

print(f"\nFinal model saved at: {final_model_path}")
print("Training with Llama 3 completed successfully!")

# Embedding Drift Functions

In [ ]:
import numpy as np
from scipy import stats
from tqdm import tqdm
from sklearn.metrics import silhouette_score, roc_auc_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.stats import gaussian_kde, ks_2samp, mannwhitneyu, anderson_ksamp
from scipy.stats import entropy, wasserstein_distance
import warnings
import time

# Try to import numba for JIT compilation
try:
    from numba import jit, prange
    NUMBA_AVAILABLE = True
except ImportError:
    NUMBA_AVAILABLE = False
    # Create dummy decorators if numba is not available
    def jit(func=None, **kwargs):
        def decorator(f):
            return f
        return decorator(func) if func else decorator

    def prange(x):
        return range(x)

# Try to import seaborn and find_peaks
try:
    from scipy.signal import find_peaks
except ImportError:
    def find_peaks(data):
        """Simple peak finding fallback"""
        peaks = []
        for i in range(1, len(data)-1):
            if data[i] > data[i-1] and data[i] > data[i+1]:
                peaks.append(i)
        return peaks, {}

try:
    import seaborn as sns
except ImportError:
    sns = None

# Original drift calculation (for compatibility)
def calculate_embedding_drift(A, B):
    return 1 - (np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B)))

# Optimized drift calculation functions
@jit(nopython=True, parallel=True)
def calculate_embedding_drift_vectorized_numba(A, B):
    """Numba-optimized vectorized drift calculation"""
    n = A.shape[0]
    drift_scores = np.empty(n, dtype=np.float64)

    for i in prange(n):
        dot_product = 0.0
        norm_a = 0.0
        norm_b = 0.0

        for j in range(A.shape[1]):
            a_val = A[i, j]
            b_val = B[i, j]
            dot_product += a_val * b_val
            norm_a += a_val * a_val
            norm_b += b_val * b_val

        norm_a = np.sqrt(norm_a)
        norm_b = np.sqrt(norm_b)

        if norm_a > 0 and norm_b > 0:
            cosine_sim = dot_product / (norm_a * norm_b)
            drift_scores[i] = 1.0 - cosine_sim
        else:
            drift_scores[i] = 1.0

    return drift_scores

def calculate_embedding_drift_vectorized_numpy(A, B):
    """Vectorized numpy implementation for drift calculation"""
    # Normalize vectors
    A_norm = np.linalg.norm(A, axis=1, keepdims=True)
    B_norm = np.linalg.norm(B, axis=1, keepdims=True)

    # Handle zero vectors
    A_norm = np.where(A_norm == 0, 1, A_norm)
    B_norm = np.where(B_norm == 0, 1, B_norm)

    A_normalized = A / A_norm
    B_normalized = B / B_norm

    # Calculate cosine similarity using dot product
    cosine_similarities = np.sum(A_normalized * B_normalized, axis=1)

    # Convert to drift scores
    drift_scores = 1 - cosine_similarities

    return drift_scores

def calculate_additional_distance_metrics(A, B):
    """Calculate multiple distance/similarity metrics between embeddings"""
    # Euclidean distance
    euclidean_dist = np.linalg.norm(A - B)

    # Manhattan distance
    manhattan_dist = np.sum(np.abs(A - B))

    # Chebyshev distance (L-infinity norm)
    chebyshev_dist = np.max(np.abs(A - B))

    # Mahalanobis-like distance (simplified)
    combined = np.vstack([A.reshape(1, -1), B.reshape(1, -1)])
    cov = np.cov(combined.T) + np.eye(len(A)) * 1e-6
    try:
        inv_cov = np.linalg.inv(cov)
        diff = A - B
        mahalanobis_dist = np.sqrt(diff.T @ inv_cov @ diff)
    except:
        mahalanobis_dist = euclidean_dist

    # Jensen-Shannon divergence (for normalized embeddings)
    A_norm = np.abs(A) / (np.sum(np.abs(A)) + 1e-8)
    B_norm = np.abs(B) / (np.sum(np.abs(B)) + 1e-8)
    M = 0.5 * (A_norm + B_norm)
    js_div = 0.5 * entropy(A_norm, M) + 0.5 * entropy(B_norm, M)

    return {
        'euclidean': euclidean_dist,
        'manhattan': manhattan_dist,
        'chebyshev': chebyshev_dist,
        'mahalanobis': mahalanobis_dist,
        'js_divergence': js_div
    }

def calculate_additional_distance_metrics_vectorized(A, B):
    """Vectorized calculation of multiple distance metrics"""
    n_samples = A.shape[0]

    # Pre-allocate results dictionary
    results = {
        'euclidean': np.empty(n_samples),
        'manhattan': np.empty(n_samples),
        'chebyshev': np.empty(n_samples),
        'mahalanobis': np.empty(n_samples),
        'js_divergence': np.empty(n_samples)
    }

    # Euclidean distance - vectorized
    diff = A - B
    results['euclidean'] = np.linalg.norm(diff, axis=1)

    # Manhattan distance - vectorized
    results['manhattan'] = np.sum(np.abs(diff), axis=1)

    # Chebyshev distance - vectorized
    results['chebyshev'] = np.max(np.abs(diff), axis=1)

    # For Mahalanobis - use simplified approach for speed
    try:
        # Use identity matrix approximation for speed
        results['mahalanobis'] = results['euclidean'].copy()
    except:
        results['mahalanobis'] = results['euclidean'].copy()

    # JS divergence calculation - vectorized where possible
    A_abs = np.abs(A)
    B_abs = np.abs(B)
    A_sum = np.sum(A_abs, axis=1, keepdims=True) + 1e-8
    B_sum = np.sum(B_abs, axis=1, keepdims=True) + 1e-8

    A_norm = A_abs / A_sum
    B_norm = B_abs / B_sum
    M = 0.5 * (A_norm + B_norm)

    # Calculate JS divergence for each sample
    js_divs = np.empty(n_samples)
    for i in range(n_samples):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                js_divs[i] = 0.5 * entropy(A_norm[i], M[i]) + 0.5 * entropy(B_norm[i], M[i])
        except:
            js_divs[i] = 0.0

    results['js_divergence'] = js_divs

    return results

def calculate_distance_metrics_batch(A, B, batch_size=1000):
    """Process distance metrics in batches to manage memory"""
    n_samples = A.shape[0]
    n_batches = (n_samples + batch_size - 1) // batch_size

    # Initialize result arrays
    euclidean = np.empty(n_samples)
    manhattan = np.empty(n_samples)
    chebyshev = np.empty(n_samples)
    mahalanobis = np.empty(n_samples)
    js_divergence = np.empty(n_samples)

    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_samples)

        A_batch = A[start_idx:end_idx]
        B_batch = B[start_idx:end_idx]

        batch_results = calculate_additional_distance_metrics_vectorized(A_batch, B_batch)

        euclidean[start_idx:end_idx] = batch_results['euclidean']
        manhattan[start_idx:end_idx] = batch_results['manhattan']
        chebyshev[start_idx:end_idx] = batch_results['chebyshev']
        mahalanobis[start_idx:end_idx] = batch_results['mahalanobis']
        js_divergence[start_idx:end_idx] = batch_results['js_divergence']

    return {
        'euclidean': euclidean,
        'manhattan': manhattan,
        'chebyshev': chebyshev,
        'mahalanobis': mahalanobis,
        'js_divergence': js_divergence
    }

# Threshold finding methods - all with proper error handling
def find_threshold_gaussian_mixture(drift_scores, n_components=2):
    """Use Gaussian Mixture Model to find natural separation point"""
    try:
        drift_array = np.array(drift_scores).reshape(-1, 1)
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(drift_array)

        means = gmm.means_.flatten()
        weights = gmm.weights_
        covariances = gmm.covariances_.flatten()

        if n_components == 2:
            mu1, mu2 = sorted(means)
            sigma1, sigma2 = np.sqrt(covariances[np.argsort(means)])
            w1, w2 = weights[np.argsort(means)]

            if sigma1 != sigma2:
                a = 1/(2*sigma1**2) - 1/(2*sigma2**2)
                b = mu2/(sigma2**2) - mu1/(sigma1**2)
                c = mu1**2/(2*sigma1**2) - mu2**2/(2*sigma2**2) - np.log((sigma2*w1)/(sigma1*w2))

                discriminant = b**2 - 4*a*c
                if discriminant >= 0:
                    threshold = (-b + np.sqrt(discriminant)) / (2*a)
                else:
                    threshold = (mu1 + mu2) / 2
            else:
                threshold = (mu1 + mu2) / 2 + (sigma1**2) * np.log(w2/w1) / (mu2 - mu1)

            threshold = max(mu1, min(mu2, threshold))
        else:
            sorted_means = np.sort(means)
            threshold = (sorted_means[-1] + sorted_means[-2]) / 2

        return threshold, gmm
    except Exception as e:
        return np.percentile(drift_scores, 90), None

def find_threshold_isolation_forest(drift_scores, contamination=0.1):
    """Use Isolation Forest to find anomalies in drift scores"""
    try:
        iso_forest = IsolationForest(contamination=contamination, random_state=42)
        scores_reshaped = np.array(drift_scores).reshape(-1, 1)
        predictions = iso_forest.fit_predict(scores_reshaped)

        threshold_idx = np.where(predictions == 1)[0]
        if len(threshold_idx) > 0:
            min_inlier_score = np.min(drift_scores[threshold_idx])
            max_inlier_score = np.max(drift_scores[threshold_idx])
            threshold = (min_inlier_score + max_inlier_score) / 2
        else:
            threshold = np.percentile(drift_scores, 90)

        return threshold
    except Exception as e:
        return np.percentile(drift_scores, 90)

def find_threshold_lof(drift_scores, n_neighbors=20, contamination=0.1):
    """Use Local Outlier Factor to find threshold"""
    try:
        # Ensure reasonable neighbor count
        n_neighbors = min(n_neighbors, len(drift_scores) // 4, 50)
        n_neighbors = max(n_neighbors, 2)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            lof = LocalOutlierFactor(n_neighbors=n_neighbors, contamination=contamination)
            scores_reshaped = np.array(drift_scores).reshape(-1, 1)
            predictions = lof.fit_predict(scores_reshaped)

            inlier_scores = drift_scores[predictions == 1]
            if len(inlier_scores) > 0:
                threshold = np.max(inlier_scores)
            else:
                threshold = np.percentile(drift_scores, 90)

        return threshold
    except Exception as e:
        return np.percentile(drift_scores, 90)

def find_threshold_isolation_based(drift_scores, contamination_estimate=0.1):
    """Use isolation-based approach assuming a certain contamination rate"""
    try:
        sorted_scores = np.sort(drift_scores)
        threshold_idx = int(len(sorted_scores) * (1 - contamination_estimate))
        threshold = sorted_scores[threshold_idx]
        return threshold
    except Exception as e:
        return np.percentile(drift_scores, 90)

def find_threshold_knee_detection(drift_scores):
    """Find threshold using knee detection on sorted drift scores"""
    try:
        sorted_scores = np.sort(drift_scores)
        n = len(sorted_scores)

        first_point = np.array([0, sorted_scores[0]])
        last_point = np.array([n-1, sorted_scores[-1]])

        max_distance = 0
        knee_idx = 0

        line_vec = last_point - first_point
        line_norm = np.linalg.norm(line_vec)

        if line_norm == 0:
            return sorted_scores[n//2]

        for i in range(1, n-1):
            point = np.array([i, sorted_scores[i]])
            point_vec = point - first_point

            # Calculate distance using cross product
            cross_product = line_vec[0] * point_vec[1] - line_vec[1] * point_vec[0]
            distance = abs(cross_product) / line_norm

            if distance > max_distance:
                max_distance = distance
                knee_idx = i

        return sorted_scores[knee_idx]
    except Exception as e:
        return np.percentile(drift_scores, 90)

def find_threshold_density_based(drift_scores):
    """Find threshold using density-based approach"""
    try:
        kde = gaussian_kde(drift_scores)

        score_min, score_max = np.min(drift_scores), np.max(drift_scores)
        x_grid = np.linspace(score_min, score_max, 1000)
        density = kde(x_grid)

        valleys, _ = find_peaks(-density)

        if len(valleys) > 0:
            threshold = x_grid[valleys[-1]]
        else:
            threshold = np.percentile(drift_scores, 90)

        return threshold
    except Exception as e:
        return np.percentile(drift_scores, 90)

# Statistical analysis functions
def calculate_distribution_metrics(flagged_scores, normal_scores):
    """Calculate various distribution comparison metrics"""
    metrics = {}

    if len(flagged_scores) == 0 or len(normal_scores) == 0:
        return {
            'ks_statistic': 0, 'ks_pvalue': 1,
            'mann_whitney_u': 0, 'mann_whitney_p': 1,
            'wasserstein_distance': 0,
            'anderson_statistic': 0, 'anderson_pvalue': 1,
            'variance_ratio': 1,
            'skewness_diff': 0,
            'kurtosis_diff': 0
        }

    # Kolmogorov-Smirnov test
    try:
        ks_stat, ks_p = ks_2samp(normal_scores, flagged_scores)
        metrics['ks_statistic'] = ks_stat
        metrics['ks_pvalue'] = ks_p
    except Exception:
        metrics['ks_statistic'] = 0
        metrics['ks_pvalue'] = 1

    # Mann-Whitney U test
    try:
        mw_stat, mw_p = mannwhitneyu(normal_scores, flagged_scores, alternative='two-sided')
        metrics['mann_whitney_u'] = mw_stat
        metrics['mann_whitney_p'] = mw_p
    except Exception:
        metrics['mann_whitney_u'] = 0
        metrics['mann_whitney_p'] = 1

    # Wasserstein distance
    try:
        metrics['wasserstein_distance'] = wasserstein_distance(normal_scores, flagged_scores)
    except Exception:
        metrics['wasserstein_distance'] = 0

    # Anderson-Darling test with proper error handling
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ad_result = anderson_ksamp([normal_scores, flagged_scores])
            metrics['anderson_statistic'] = ad_result.statistic
            # Handle p-value floor warning by setting minimum
            metrics['anderson_pvalue'] = max(0.001, getattr(ad_result, 'significance_level', 1))
    except Exception:
        metrics['anderson_statistic'] = 0
        metrics['anderson_pvalue'] = 1

    # Distribution shape metrics
    try:
        metrics['variance_ratio'] = np.var(flagged_scores) / (np.var(normal_scores) + 1e-8)
        metrics['skewness_diff'] = stats.skew(flagged_scores) - stats.skew(normal_scores)
        metrics['kurtosis_diff'] = stats.kurtosis(flagged_scores) - stats.kurtosis(normal_scores)
    except Exception:
        metrics['variance_ratio'] = 1
        metrics['skewness_diff'] = 0
        metrics['kurtosis_diff'] = 0

    return metrics

def calculate_clustering_metrics(drift_scores, flags):
    """Calculate additional clustering quality metrics"""
    if np.sum(flags) == 0 or np.sum(flags) == len(flags):
        return {
            'calinski_harabasz': 0,
            'davies_bouldin': float('inf'),
            'intra_cluster_variance_normal': 0,
            'intra_cluster_variance_flagged': 0,
            'inter_cluster_distance': 0
        }

    X = drift_scores.reshape(-1, 1)
    labels = flags.astype(int)

    try:
        ch_score = calinski_harabasz_score(X, labels)
    except:
        ch_score = 0

    try:
        db_score = davies_bouldin_score(X, labels)
    except:
        db_score = float('inf')

    normal_scores = drift_scores[~flags]
    flagged_scores = drift_scores[flags]

    intra_var_normal = np.var(normal_scores) if len(normal_scores) > 1 else 0
    intra_var_flagged = np.var(flagged_scores) if len(flagged_scores) > 1 else 0

    mean_normal = np.mean(normal_scores) if len(normal_scores) > 0 else 0
    mean_flagged = np.mean(flagged_scores) if len(flagged_scores) > 0 else 0
    inter_distance = abs(mean_flagged - mean_normal)

    return {
        'calinski_harabasz': ch_score,
        'davies_bouldin': db_score,
        'intra_cluster_variance_normal': intra_var_normal,
        'intra_cluster_variance_flagged': intra_var_flagged,
        'inter_cluster_distance': inter_distance
    }

def calculate_stability_metrics(drift_scores, threshold, bootstrap_samples=100):
    """Calculate threshold stability through bootstrap sampling"""
    n_samples = len(drift_scores)
    bootstrap_flags = []

    np.random.seed(42)

    try:
        for _ in range(bootstrap_samples):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            boot_scores = drift_scores[indices]
            boot_flags = boot_scores > threshold
            bootstrap_flags.append(np.mean(boot_flags))

        if len(bootstrap_flags) == 0:
            return {
                'bootstrap_variance': 0,
                'bootstrap_std': 0,
                'bootstrap_cv': 0,
                'bootstrap_samples': bootstrap_samples
            }

        stability_variance = np.var(bootstrap_flags)
        stability_std = np.std(bootstrap_flags)
        mean_flag_rate = np.mean(bootstrap_flags)
        stability_cv = stability_std / (mean_flag_rate + 1e-8)

        return {
            'bootstrap_variance': stability_variance,
            'bootstrap_std': stability_std,
            'bootstrap_cv': stability_cv,
            'bootstrap_samples': bootstrap_samples
        }
    except Exception as e:
        return {
            'bootstrap_variance': 0,
            'bootstrap_std': 0,
            'bootstrap_cv': 0,
            'bootstrap_samples': bootstrap_samples
        }

def evaluate_threshold_unsupervised(drift_scores, threshold):
    """Enhanced evaluation with additional metrics"""
    flags = np.array(drift_scores) > threshold

    if np.sum(flags) == 0 or np.sum(flags) == len(flags):
        return {
        'threshold': threshold,
        'flagged_count': np.sum(flags),
        'flagged_percentage': np.mean(flags) * 100,
        'silhouette_score': -1,
        'separation_ratio': 0,
        'quality_score': 0,
        'mann_whitney_p': 1,  # default so no KeyError
        'calinski_harabasz': 0,
        'bootstrap_cv': 1
      }

    flagged_scores = np.array(drift_scores)[flags]
    normal_scores = np.array(drift_scores)[~flags]

    mean_flagged = np.mean(flagged_scores) if len(flagged_scores) > 0 else 0
    mean_normal = np.mean(normal_scores) if len(normal_scores) > 0 else 0

    std_flagged = np.std(flagged_scores) if len(flagged_scores) > 1 else 0
    std_normal = np.std(normal_scores) if len(normal_scores) > 1 else 0
    pooled_std = np.sqrt(((len(flagged_scores)-1)*std_flagged**2 + (len(normal_scores)-1)*std_normal**2) /
                        (len(flagged_scores) + len(normal_scores) - 2))

    separation_ratio = abs(mean_flagged - mean_normal) / (pooled_std + 1e-8)

    labels = flags.astype(int)
    try:
        sil_score = silhouette_score(np.array(drift_scores).reshape(-1, 1), labels)
    except:
        sil_score = 0

    flagged_pct = np.mean(flags) * 100
    rate_penalty = abs(flagged_pct - 50) / 25
    quality_score = sil_score + separation_ratio - rate_penalty

    cohens_d = (mean_flagged - mean_normal) / (pooled_std + 1e-8)

    try:
        auc_score = roc_auc_score(flags.astype(int), drift_scores)
    except:
        auc_score = 0

    percentile_gap = np.median(flagged_scores) - np.median(normal_scores)

    p_flagged = np.mean(flags)
    flag_entropy = entropy([p_flagged, 1 - p_flagged], base=2) if p_flagged > 0 and p_flagged < 1 else 0

    max_z_score = np.max(stats.zscore(drift_scores)[flags]) if np.any(flags) else 0

    dist_metrics = calculate_distribution_metrics(flagged_scores, normal_scores)
    cluster_metrics = calculate_clustering_metrics(np.array(drift_scores), flags)
    stability_metrics = calculate_stability_metrics(np.array(drift_scores), threshold)

    sorted_indices = np.argsort(drift_scores)[::-1]
    precision_at_k = {}
    for k in [100, 500, 1000, 5000]:
        if k < len(drift_scores):
            top_k_flags = flags[sorted_indices[:k]]
            precision_at_k[f'precision_at_{k}'] = np.mean(top_k_flags)

    try:
        sorted_scores = np.sort(drift_scores)
        n = len(sorted_scores)
        if n > 0 and np.sum(sorted_scores) > 0:
            gini = (2 * np.sum((np.arange(n) + 1) * sorted_scores)) / (n * np.sum(sorted_scores)) - (n + 1) / n
        else:
            gini = 0
    except Exception:
        gini = 0

    try:
        hist, _ = np.histogram(drift_scores, bins=50)
        hist_norm = hist / np.sum(hist)
        hist_norm = hist_norm[hist_norm > 0]
        if len(hist_norm) > 0:
            score_entropy = entropy(hist_norm)
        else:
            score_entropy = 0
    except Exception:
        score_entropy = 0

    result = {
        'threshold': threshold,
        'flagged_count': np.sum(flags),
        'flagged_percentage': flagged_pct,
        'silhouette_score': sil_score,
        'separation_ratio': separation_ratio,
        'quality_score': quality_score,
        'mean_flagged': mean_flagged,
        'mean_normal': mean_normal,
        'cohens_d': cohens_d,
        'auc_score': auc_score,
        'percentile_gap': percentile_gap,
        'flag_entropy': flag_entropy,
        'max_z_score': max_z_score,
        'median_flagged': np.median(flagged_scores),
        'median_normal': np.median(normal_scores),
        'std_flagged': std_flagged,
        'std_normal': std_normal,
        'gini_coefficient': gini,
        'score_entropy': score_entropy,
        **dist_metrics,
        **cluster_metrics,
        **stability_metrics,
        **precision_at_k
    }

    return result

# Main optimized function
def batch_drift_detection_unsupervised_optimized(dataset_dict, model=None, threshold_methods=['auto'],
                                               plot_results=True, batch_size=1000, use_numba=None):
    """
    Optimized version of batch drift detection with significant performance improvements
    """

    if model is None:
        try:
            model = globals()['model']
        except KeyError:
            raise ValueError("Model must be provided either as parameter or set as global variable 'model'")

    if use_numba is None:
        use_numba = NUMBA_AVAILABLE

    if use_numba and NUMBA_AVAILABLE:
        print("Using Numba optimization for drift calculation")
        drift_calc_func = calculate_embedding_drift_vectorized_numba
    else:
        print("Using NumPy vectorized optimization for drift calculation")
        drift_calc_func = calculate_embedding_drift_vectorized_numpy

    available_methods = {
        'gaussian_mixture': 'Gaussian Mixture Model (assumes 2 populations)',
        'isolation_forest_10': 'Isolation Forest (10% contamination)',
        'isolation_forest_15': 'Isolation Forest (15% contamination)',
        'isolation_forest_5': 'Isolation Forest (5% contamination)',
        'lof': 'Local Outlier Factor',
        'isolation_10': 'Isolation-based (10% contamination)',
        'isolation_15': 'Isolation-based (15% contamination)',
        'isolation_5': 'Isolation-based (5% contamination)',
        'knee_detection': 'Knee detection on sorted scores',
        'density_valley': 'Density-based valley detection',
        'mad': 'Median Absolute Deviation',
        'iqr': 'Interquartile Range',
        'percentile_95': '95th percentile',
        'percentile_90': '90th percentile',
        'percentile_75': '75th percentile',
        'percentile_60': '60th percentile',
        'percentile_55': '55th percentile',
        'percentile_50': '50th percentile (median)',
        'z_score_2': 'Z-score > 2',
        'z_score_1_5': 'Z-score > 1.5',
        'modified_z_score': 'Modified Z-score (MAD-based)',
        'tukey_outlier': 'Tukey outlier detection',
        'grubbs_test': 'Grubbs test adaptation'
    }

    if threshold_methods == ['auto']:
        threshold_methods = list(available_methods.keys())

    optimal_thresholds = {}
    threshold_analysis = {}

    for split_name, dataset in dataset_dict.items():
        print(f"\nProcessing split: {split_name}")

        # Generate embeddings with progress bar
        print("Encoding body texts...")
        body_embeddings = model.encode(list(dataset["body"]), show_progress_bar=True)
        print("Encoding pair texts...")
        pair_embeddings = model.encode(list(dataset['pair']), show_progress_bar=True)

        # Convert to numpy arrays for faster computation
        body_embeddings = np.array(body_embeddings, dtype=np.float32)
        pair_embeddings = np.array(pair_embeddings, dtype=np.float32)

        print(f"Computing drift scores for {len(body_embeddings)} samples...")

        # Calculate drift scores - OPTIMIZED
        start_time = time.time()
        drift_scores = drift_calc_func(body_embeddings, pair_embeddings)

        # Calculate additional distance metrics in batches - OPTIMIZED
        print("Computing additional distance metrics...")
        additional_distances_dict = calculate_distance_metrics_batch(
            body_embeddings, pair_embeddings, batch_size=batch_size
        )

        # Convert to list of dictionaries for compatibility with existing code
        additional_distances = []
        for i in range(len(drift_scores)):
            distances = {
                metric: additional_distances_dict[metric][i]
                for metric in additional_distances_dict.keys()
            }
            additional_distances.append(distances)

        elapsed = time.time() - start_time
        print(f"Distance calculations completed in {elapsed:.2f} seconds")

        print(f"Drift scores - Mean: {np.mean(drift_scores):.4f}, Std: {np.std(drift_scores):.4f}")
        print(f"Drift scores - Min: {np.min(drift_scores):.4f}, Max: {np.max(drift_scores):.4f}")
        print(f"Drift scores - 50th percentile: {np.percentile(drift_scores, 50):.4f}")

        # Test different threshold methods
        method_results = {}

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            for method in threshold_methods:
                try:
                    if method == 'gaussian_mixture':
                        threshold, _ = find_threshold_gaussian_mixture(drift_scores)
                    elif method.startswith('isolation_forest_'):
                        contamination = int(method.split('_')[2]) / 100
                        threshold = find_threshold_isolation_forest(drift_scores, contamination)
                    elif method == 'lof':
                        threshold = find_threshold_lof(drift_scores)
                    elif method.startswith('isolation_'):
                        contamination = int(method.split('_')[1]) / 100
                        threshold = find_threshold_isolation_based(drift_scores, contamination)
                    elif method == 'knee_detection':
                        threshold = find_threshold_knee_detection(drift_scores)
                    elif method == 'density_valley':
                        threshold = find_threshold_density_based(drift_scores)
                    elif method == 'mad':
                        median = np.median(drift_scores)
                        mad = np.median(np.abs(drift_scores - median))
                        threshold = median + 2.5 * mad
                    elif method == 'iqr':
                        q1, q3 = np.percentile(drift_scores, [25, 75])
                        iqr = q3 - q1
                        threshold = q3 + 1.5 * iqr
                    elif method.startswith('percentile_'):
                        pct = int(method.split('_')[1])
                        threshold = np.percentile(drift_scores, pct)
                    elif method == 'z_score_2':
                        mean_score = np.mean(drift_scores)
                        std_score = np.std(drift_scores)
                        threshold = mean_score + 2 * std_score
                    elif method == 'z_score_1_5':
                        mean_score = np.mean(drift_scores)
                        std_score = np.std(drift_scores)
                        threshold = mean_score + 1.5 * std_score
                    elif method == 'modified_z_score':
                        median = np.median(drift_scores)
                        mad = np.median(np.abs(drift_scores - median))
                        threshold = median + 3.5 * mad
                    elif method == 'tukey_outlier':
                        q1, q3 = np.percentile(drift_scores, [25, 75])
                        iqr = q3 - q1
                        threshold = q3 + 3.0 * iqr
                    elif method == 'grubbs_test':
                        mean_score = np.mean(drift_scores)
                        std_score = np.std(drift_scores)
                        n = len(drift_scores)
                        try:
                            t_val = stats.t.ppf(1-0.05/(2*n), n-2)
                            g_critical = (n-1)/np.sqrt(n) * np.sqrt(t_val**2 / (n-2+t_val**2))
                            threshold = mean_score + g_critical * std_score
                        except Exception:
                            threshold = mean_score + 2.5 * std_score
                    else:
                        threshold = np.median(drift_scores)

                    # Evaluate this threshold
                    evaluation = evaluate_threshold_unsupervised(drift_scores, threshold)
                    evaluation['method'] = method
                    evaluation['description'] = available_methods.get(method, method)
                    method_results[method] = evaluation

                except Exception as e:
                    print(f"Warning: Method {method} failed: {str(e)}")
                    continue

        # Select best method
        def enhanced_quality_score(result):
            base_score = result['quality_score']

            if result['mann_whitney_p'] < 0.001:
                base_score += 0.5
            elif result['mann_whitney_p'] < 0.01:
                base_score += 0.3
            elif result['mann_whitney_p'] < 0.05:
                base_score += 0.1

            if result['calinski_harabasz'] > 100:
                base_score += 0.2

            flag_pct = result['flagged_percentage']
            if flag_pct < 30 or flag_pct > 70:
                base_score -= 0.5

            if result['bootstrap_cv'] < 0.1:
                base_score += 0.2

            return base_score

        if method_results:
            best_method = max(method_results.keys(), key=lambda x: enhanced_quality_score(method_results[x]))
            optimal_threshold = method_results[best_method]['threshold']
        else:
            best_method = 'percentile_90'
            optimal_threshold = np.percentile(drift_scores, 90)

        optimal_thresholds[split_name] = optimal_threshold

        # Apply optimal threshold
        flags = drift_scores > optimal_threshold
        z_scores = stats.zscore(drift_scores)

        # Remove existing columns and add new ones
        for col in ['flagged', 'drift_scores', 'z_scores', 'optimal_threshold']:
            if col in dataset.column_names:
                dataset = dataset.remove_columns(col)

        dataset = dataset.add_column('flagged', flags.tolist())
        dataset = dataset.add_column('drift_scores', drift_scores.tolist())
        dataset = dataset.add_column('z_scores', z_scores.tolist())
        dataset = dataset.add_column('optimal_threshold', [optimal_threshold] * len(drift_scores))

        # Add additional distance metrics as columns
        for metric_name in additional_distances_dict.keys():
            dataset = dataset.add_column(f'{metric_name}_distance',
                                       additional_distances_dict[metric_name].tolist())

        dataset_dict[split_name] = dataset

        # Store analysis
        threshold_analysis[split_name] = {
            'selected_method': best_method,
            'all_methods': method_results,
            'drift_scores_stats': {
                'mean': np.mean(drift_scores),
                'std': np.std(drift_scores),
                'min': np.min(drift_scores),
                'max': np.max(drift_scores),
                'median': np.median(drift_scores),
                'q1': np.percentile(drift_scores, 25),
                'q3': np.percentile(drift_scores, 75),
                'skewness': stats.skew(drift_scores),
                'kurtosis': stats.kurtosis(drift_scores)
            },
            'distance_metrics_stats': {
                metric: {
                    'mean': np.mean(additional_distances_dict[metric]),
                    'std': np.std(additional_distances_dict[metric]),
                    'correlation_with_drift': np.corrcoef(drift_scores, additional_distances_dict[metric])[0,1]
                } for metric in additional_distances_dict.keys()
            }
        }

        # Print results
        if method_results and best_method in method_results:
            best_result = method_results[best_method]
            print(f"\nEnhanced Results for {split_name}:")
            print(f"Selected method: {best_method} - {available_methods.get(best_method, best_method)}")
            print(f"Optimal threshold: {optimal_threshold:.4f}")
            print(f"Flagged samples: {np.sum(flags)} out of {len(flags)} ({np.mean(flags)*100:.1f}%)")
            print(f"Enhanced quality score: {enhanced_quality_score(best_result):.3f}")
            print(f"Statistical significance (Mann-Whitney p): {best_result['mann_whitney_p']:.2e}")
            print(f"Effect size (Cohen's d): {best_result['cohens_d']:.3f}")
            print(f"Wasserstein distance: {best_result['wasserstein_distance']:.4f}")
            print(f"Bootstrap stability (CV): {best_result['bootstrap_cv']:.3f}")
            print(f"Calinski-Harabasz score: {best_result['calinski_harabasz']:.1f}")

            # Show comparison of top methods
            print(f"\nTop 5 Method Comparison:")
            sorted_methods = sorted(method_results.items(),
                                  key=lambda x: enhanced_quality_score(x[1]), reverse=True)
            for i, (method, result) in enumerate(sorted_methods[:5]):
                print(f"  {i+1}. {method}: Enhanced={enhanced_quality_score(result):.3f}, "
                      f"Flag%={result['flagged_percentage']:.1f}, "
                      f"Cohen's d={result['cohens_d']:.3f}, "
                      f"p-val={result['mann_whitney_p']:.2e}")
        else:
            print(f"\nResults for {split_name}:")
            print(f"Used fallback threshold: {optimal_threshold:.4f}")
            print(f"Flagged samples: {np.sum(flags)} out of {len(flags)} ({np.mean(flags)*100:.1f}%)")

    return dataset_dict, optimal_thresholds, threshold_analysis

# Utility functions
def print_performance_tips():
    """Print performance optimization recommendations"""
    print("\n🔧 PERFORMANCE OPTIMIZATION TIPS:")
    print("=" * 40)
    print("1. 🎯 Batch Size:")
    print("   • Larger batch_size = faster but more memory")
    print("   • Try: 500 (low memory), 1000 (balanced), 2000+ (high memory)")

    print("\n2. ⚡ Numba Installation:")
    if NUMBA_AVAILABLE:
        print("   ✅ Numba is installed and active")
    else:
        print("   ❌ Install numba for 2-5x speedup: pip install numba")

    print("\n3. 🧠 Memory Management:")
    print("   • Close other applications")
    print("   • Use smaller batch sizes if getting memory errors")
    print("   • Monitor memory usage during processing")

    print("\n4. 🔄 Method Selection:")
    print("   • Use fewer threshold_methods for faster processing")
    print("   • Best performing methods: ['gaussian_mixture', 'isolation_10', 'percentile_90']")

def quick_performance_check(n_samples=1000, n_dims=384):
    """Quick performance check with synthetic data"""
    print(f"\n🧪 QUICK PERFORMANCE CHECK")
    print(f"Testing with {n_samples:,} samples, {n_dims} dimensions")
    print("-" * 40)

    # Generate synthetic embeddings
    np.random.seed(42)
    body_embeddings = np.random.randn(n_samples, n_dims).astype(np.float32)
    pair_embeddings = np.random.randn(n_samples, n_dims).astype(np.float32)

    # Test different methods
    methods = [
        ("NumPy Vectorized", calculate_embedding_drift_vectorized_numpy)
    ]

    if NUMBA_AVAILABLE:
        methods.append(("Numba JIT", calculate_embedding_drift_vectorized_numba))

    for name, method in methods:
        start_time = time.time()
        scores = method(body_embeddings, pair_embeddings)
        elapsed = time.time() - start_time

        rate = n_samples / elapsed
        print(f"{name:15}: {elapsed:.3f}s ({rate:.0f} samples/sec)")

    print(f"\nEstimated time for 50k samples:")
    for name, method in methods:
        estimated_time = 50000 / rate  # Use last calculated rate
        print(f"{name:15}: ~{estimated_time:.1f} seconds")

def run_optimized_drift_analysis(dataset_dict, model, use_numba=None, batch_size=1000,
                                suppress_warnings=True):
    """
    Run the optimized drift detection analysis with clean output

    Args:
        dataset_dict: Dictionary containing your datasets
        model: The sentence transformer model
        use_numba: Whether to use Numba optimization (None for auto-detect)
        batch_size: Batch size for processing distance metrics
        suppress_warnings: Whether to suppress warning messages
    """

    if suppress_warnings:
        import warnings
        warnings.filterwarnings('ignore')

    # Set global model variable
    globals()['model'] = model

    print("🚀 Starting Optimized Drift Analysis")
    print("=" * 50)

    total_samples = sum(len(dataset) for dataset in dataset_dict.values())
    print(f"📊 Total samples across all splits: {total_samples:,}")

    start_time = time.time()

    # Run the optimized analysis
    try:
        updated_datasets, thresholds, analysis = batch_drift_detection_unsupervised_optimized(
            dataset_dict,
            model=model,
            threshold_methods=['auto'],
            plot_results=False,
            use_numba=use_numba,
            batch_size=batch_size
        )

        total_time = time.time() - start_time

        print("\n" + "=" * 50)
        print("✅ ANALYSIS COMPLETE!")
        print(f"⏱️  Total processing time: {total_time:.2f} seconds")
        print(f"📈 Processing rate: {total_samples/total_time:.1f} samples/second")

        # Summary statistics
        print(f"\n📋 SUMMARY:")
        for split_name in dataset_dict.keys():
            if split_name in analysis:
                method_info = analysis[split_name]['all_methods']
                selected_method = analysis[split_name]['selected_method']
                if selected_method in method_info:
                    flagged_count = method_info[selected_method]['flagged_count']
                    total_count = len(dataset_dict[split_name])
                    flagged_pct = (flagged_count / total_count) * 100
                    print(f"   {split_name}: {flagged_count:,}/{total_count:,} flagged ({flagged_pct:.1f}%)")

        if suppress_warnings:
            warnings.resetwarnings()

        return updated_datasets, thresholds, analysis

    except Exception as e:
        print(f"❌ Error during analysis: {str(e)}")
        print("🔧 Try reducing batch_size or disabling numba")
        if suppress_warnings:
            warnings.resetwarnings()
        raise

# Simple benchmark function
def benchmark_drift_calculation(body_embeddings, pair_embeddings, n_runs=3):
    """Benchmark different drift calculation methods"""

    print("Benchmarking drift calculation methods...")
    print(f"Testing with {len(body_embeddings)} samples, {body_embeddings.shape[1]} dimensions")

    methods = {}

    # Original method (single calculation)
    def original_method(A, B):
        drift_scores = []
        for i in range(len(A)):
            score = calculate_embedding_drift(A[i], B[i])
            drift_scores.append(score)
        return np.array(drift_scores)

    methods['Original (loop)'] = original_method
    methods['NumPy vectorized'] = calculate_embedding_drift_vectorized_numpy

    if NUMBA_AVAILABLE:
        methods['Numba optimized'] = calculate_embedding_drift_vectorized_numba

    results = {}

    for name, method in methods.items():
        times = []
        for run in range(n_runs):
            start_time = time.time()
            scores = method(body_embeddings, pair_embeddings)
            end_time = time.time()
            times.append(end_time - start_time)

        avg_time = np.mean(times)
        std_time = np.std(times)
        results[name] = {
            'avg_time': avg_time,
            'std_time': std_time,
            'scores': scores
        }

        print(f"{name}: {avg_time:.4f} ± {std_time:.4f} seconds")

    # Verify all methods give same results
    base_scores = list(results.values())[0]['scores']
    for name, result in results.items():
        max_diff = np.max(np.abs(result['scores'] - base_scores))
        print(f"{name} max difference from first method: {max_diff:.10f}")

    return results

# Embedding Drift Tests

In [ ]:
test_data_flagged, optimal_thresholds, analysis = run_optimized_drift_analysis(
    dataset_dict=test_data,
    model=model,
    use_numba=True,  # Set to False to disable numba, None for auto-detect
    batch_size=1000,  # Adjust based on available memory
    suppress_warnings=True  # Clean output

)

In [ ]:
def simple_category_distribution(dataset_dict):
    """
    Simple category distribution analysis showing flagged counts and percentages

    Args:
        dataset_dict: Dictionary of datasets with drift detection results
    """

    for split_name, dataset in dataset_dict.items():
        print(f"\n{split_name}:")

        # Extract data
        categories = np.array(dataset['category'])
        flagged = np.array(dataset['flagged'])

        # Get unique categories and sort them (clean first, then alphabetically)
        unique_categories = np.unique(categories)
        if 'clean' in unique_categories:
            # Put clean first, then sort the rest
            other_categories = sorted([cat for cat in unique_categories if cat != 'clean'])
            unique_categories = ['clean'] + other_categories
        else:
            unique_categories = sorted(unique_categories)

        # Calculate and display results for each category
        for category in unique_categories:
            cat_mask = categories == category
            cat_total = np.sum(cat_mask)
            cat_flagged = np.sum(flagged[cat_mask])
            cat_flag_rate = cat_flagged / cat_total * 100

            print(f"{category}: {cat_flagged}/{cat_total} flagged ({cat_flag_rate:.1f}%)")

In [ ]:
simple_category_distribution(test_data_flagged)

In [ ]:
# INITIAL CODE
# # Number of flagged prompts to display
# x = 30  # change this as needed

# # Filter flagged prompts from the "train" split
# flagged_prompts = [
#     row["body"]
#     for row in test_data_flagged["train"]
#     if row.get("flagged") is False
# ]

# # Print the first x flagged prompt bodies
# for i, prompt in enumerate(flagged_prompts[:x], start=1):
#     print(f"{i}. {prompt}\n")
#     print(f"=====================================================================================")
#________________

def common_tokens_false_negatives(dataset_split, top_n=20, examples_per_token=3):
    """
    Find the most common tokens in prompts that were false negatives
    (flagged == False but category != 'clean'), and show example prompts.

    Args:
        dataset_split: list of dicts (a single split, e.g. test_data_flagged["train"])
        top_n: how many top tokens to display
        examples_per_token: how many example prompts to show for each token
    """
    import re
    from collections import Counter, defaultdict

    # Collect prompt bodies for false negatives
    false_negative_prompts = [
        row["body"]
        for row in dataset_split
        if (row.get("flagged") is False) and (row.get("category") != "clean")
    ]

    # Tokenize + map prompts
    token_to_prompts = defaultdict(list)
    tokens = []
    for prompt in false_negative_prompts:
        prompt_tokens = re.findall(r"\w+", prompt.lower())
        tokens.extend(prompt_tokens)
        for token in set(prompt_tokens):  # use set so we don't add same prompt multiple times
            if len(token_to_prompts[token]) < examples_per_token:
                token_to_prompts[token].append(prompt)

    # Count frequencies
    counter = Counter(tokens)

    # Display results
    print(f"\nTop {top_n} tokens in false negatives (with examples):\n")
    for token, count in counter.most_common(top_n):
        print(f"Token: '{token}'  |  Count: {count}")
        for i, ex in enumerate(token_to_prompts[token], start=1):
            print(f"   Example {i}: {ex}")
        print("-" * 90)

# --- Usage on train split ---
common_tokens_false_negatives(test_data_flagged["train"], top_n=20, examples_per_token=2)



# Visualizations

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
def visualize_embedding_space(clean_embeddings, injected_embeddings):
  all_embeddings = np.vstack([clean_embeddings,injected_embeddings])
  labels = ["Clean"] * len(clean_embeddings) + ["Injected"] * len(injected_embeddings)

  #t-SNE reduction
  tsne = TSNE(n_components = 2, random_state = 42)
  embeddings_2d = tsne.fit_transform(all_embeddings)

  plt.figure(figsize = (10,8))
  colors = ['blue', 'red']
  for i, label in enumerate(["Clean", "Injected"]):
    mask = np.array(labels) == label
    plt.scatter(embeddings_2d[mask,0], embeddings_2d[mask,1],
                c=colors[i], label=label, alpha=0.6)

  plt.legend()
  plt.title("Embedding Space Visualization (t-SNE)")
  plt.show()


In [ ]:
def plot_drift_distribution(clean_drifts, injected_drifts):
  plt.figure(figsize=(10,6))
  plt.hist(clean_drifts, bins=30, alpha=0.7, label="Clean", density=True)
  plt.hist(injected_drifts, bins=30, alpha=0.7, label="Injected", density=True)
  plt.xlabel("Cosine Distance")
  plt.ylabel("Density")
  plt.legend()
  plt.title("Distribution of Embedding Drift Scores")
  plt.show()


# Extracting prompt pairs by category for paper

In [ ]:
import pandas as pd

def display_prompt_pairs_by_category(dataset_clean_with_pairs, phase="train", num_examples=5):
    """
    Simple function to display clean and injected prompts from each category
    """
    print(f"="*80)
    print(f"PROMPT PAIRS - {phase.upper()}")
    print(f"="*80)

    # Get the specified phase
    dataset = dataset_clean_with_pairs[phase]

    # Convert to DataFrame
    data_list = []
    for i in range(len(dataset)):
        row = {}
        for feature in dataset.features:
            row[feature] = dataset[i][feature]
        data_list.append(row)

    df = pd.DataFrame(data_list)

    # Get unique categories
    categories = df['category'].unique()

    for category in categories:
        print(f"\n{'='*60}")
        print(f"CATEGORY: {category.upper()}")
        print(f"{'='*60}")

        # Get examples for this category
        category_data = df[df['category'] == category]

        # Filter valid examples (skip failed cleaning attempts)
        valid_examples = category_data[
            (category_data['pair'].notna()) &
            (~category_data['pair'].isin(['failed', 'failed_extraction', 'missing', 'api_failed']))
        ]

        if len(valid_examples) == 0:
            print(f"No valid examples found for {category}")
            continue

        # Take the first N examples
        sample_size = min(num_examples, len(valid_examples))
        sampled_examples = valid_examples.head(sample_size)

        for idx, (_, row) in enumerate(sampled_examples.iterrows(), 1):
            print(f"\n--- Example {idx} ---")
            print(f"INJECTED PROMPT:")
            print(f"{row['body']}")
            print(f"\nCLEAN PROMPT:")
            print(f"{row['pair']}")
            print(f"{'-'*40}")

# Display train prompts
display_prompt_pairs_by_category(dataset_clean_with_pairs, "train", num_examples=5)

# Testing Different Models


In [ ]:
import json
import time
import asyncio
from abc import ABC, abstractmethod
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from enum import Enum
counter = 0

class ProviderType(Enum):
    GEMINI = "gemini"
    CLAUDE = "claude"
    DEEPSEEK = "deepseek"
    QWEN = "qwen"

@dataclass
class ProviderConfig:
    name: str
    api_key: str
    models: Dict[str, str]  # task -> model_name mapping
    embeddings: Optional[str] = None
    base_url: Optional[str] = None

class APIProvider(ABC):
    def __init__(self, config: ProviderConfig):
        self.config = config
        self.client = None
        self._setup_client()

    @abstractmethod
    def _setup_client(self):
        """Initialize the API client"""
        pass

    def get_client(self):
        """Return the initialized client for use in existing functions"""
        return self.client

class GeminiProvider(APIProvider):
    def _setup_client(self):
        try:
            import google.generativeai as genai
            genai.configure(api_key=self.config.api_key)
            self.client = genai
        except ImportError:
            raise ImportError("google-generativeai package not installed. Run: pip install google-generativeai")

class ClaudeProvider(APIProvider):
    def _setup_client(self):
        try:
            import anthropic
            self.client = anthropic.Anthropic(api_key=self.config.api_key)
        except ImportError:
            raise ImportError("anthropic package not installed. Run: pip install anthropic")

class DeepseekProvider(APIProvider):
    def _setup_client(self):
        try:
            from openai import OpenAI
            # Deepseek uses OpenAI-compatible API
            self.client = OpenAI(
                api_key=self.config.api_key,
                base_url=self.config.base_url or "https://api.deepseek.com/v1"
            )
        except ImportError:
            raise ImportError("openai package not installed. Run: pip install openai")

class QwenProvider(APIProvider):
    def _setup_client(self):
        try:
            from openai import OpenAI
            # Qwen uses OpenAI-compatible API
            self.client = OpenAI(
                api_key=self.config.api_key,
                base_url=self.config.base_url or "https://dashscope.aliyuncs.com/compatible-mode/v1"
            )
        except ImportError:
            raise ImportError("openai package not installed. Run: pip install openai")

class ProviderManager:
    """Simple manager to switch between API providers"""

    def __init__(self, providers_config: Dict[str, ProviderConfig]):
        self.providers = {}
        self.current_provider_name = None

        # Initialize providers
        for provider_name, config in providers_config.items():
            self.providers[provider_name] = self._create_provider(config)

        # Set default provider
        if providers_config:
            self.current_provider_name = list(self.providers.keys())[0]

    def _create_provider(self, config: ProviderConfig) -> APIProvider:
        """Factory method to create appropriate provider"""
        if config.name.lower() == "gemini":
            return GeminiProvider(config)
        elif config.name.lower() == "claude":
            return ClaudeProvider(config)
        elif config.name.lower() == "deepseek":
            return DeepseekProvider(config)
        elif config.name.lower() == "qwen":
            return QwenProvider(config)
        else:
            raise ValueError(f"Unsupported provider: {config.name}")

    def switch_provider(self, provider_name: str):
        """Switch to a different provider"""
        if provider_name not in self.providers:
            raise ValueError(f"Provider {provider_name} not configured")
        self.current_provider_name = provider_name
        print(f"Switched to provider: {provider_name}")

    def get_current_client(self):
        """Get the current provider's client for use in your existing functions"""
        if not self.current_provider_name:
            raise ValueError("No provider selected")
        return self.providers[self.current_provider_name].get_client()

    def get_current_config(self):
        """Get current provider's configuration"""
        if not self.current_provider_name:
            raise ValueError("No provider selected")
        return self.providers[self.current_provider_name].config

    def get_current_provider_name(self):
        """Get name of current provider"""
        return self.current_provider_name

    def list_providers(self):
        """List all available providers"""
        return list(self.providers.keys())

# Simple configuration helper
def create_provider_configs():
    """Create configuration for multiple providers"""
    return {
        "gemini": ProviderConfig(
            name="gemini",
            api_key="your-gemini-key",  # Replace with actual key
            models={
                "classify": "gemini-1.5-flash",
                "clean": "gemini-1.5-pro",
                "generate": "gemini-1.5-flash"
            },
            embeddings="text-embedding-004"
        ),
        "gemini_pro": ProviderConfig(
            name="gemini",
            api_key="your-gemini-key",  # Same key, different models
            models={
                "classify": "gemini-1.5-pro",
                "clean": "gemini-1.5-pro",
                "generate": "gemini-1.5-pro"
            },
            embeddings="text-embedding-004"
        ),
        "claude": ProviderConfig(
            name="claude",
            api_key="your-claude-key",  # Replace with actual key
            models={
                "classify": "claude-3-haiku-20240307",
                "clean": "claude-3-sonnet-20240229",
                "generate": "claude-3-haiku-20240307"
            }
        ),
        "claude_sonnet": ProviderConfig(
            name="claude",
            api_key="your-claude-key",  # Same key, different models
            models={
                "classify": "claude-3-5-sonnet-20241022",
                "clean": "claude-3-5-sonnet-20241022",
                "generate": "claude-3-5-sonnet-20241022"
            }
        ),
        "deepseek": ProviderConfig(
            name="deepseek",
            api_key="your-deepseek-key",  # Replace with actual key
            base_url="https://api.deepseek.com/v1",
            models={
                "classify": "deepseek-chat",
                "clean": "deepseek-coder",
                "generate": "deepseek-chat"
            }
        ),
        "qwen": ProviderConfig(
            name="qwen",
            api_key="your-qwen-key",  # Replace with actual key
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
            models={
                "classify": "qwen-turbo",
                "clean": "qwen-plus",
                "generate": "qwen-turbo"
            }
        )
    }

# Modified versions of our existing functions that work with any provider
def create_jsonl_file_universal(batch, field, output_file="batch_requests.jsonl", provider_manager=None, task_type="classify"):
    """
    Universal version of your create_jsonl_file function that works with any provider
    """
    global counter
    text = batch[field]
    if not text:
        return "Error with Texts"

    try:
        counter += 1
        custom_id = f"batch-{counter}"

        # Get current provider config
        config = provider_manager.get_current_config()
        provider_name = provider_manager.get_current_provider_name()

        #  existing prompt templates
        if task_type == "classify":
            system_content = """You are a cybersecurity expert specializing in prompt injection classification.
                               You will receive injected prompts and must classify them into one of these attack categories:
                               1. **jailbreak** — Only prompts designed to bypass the model's safety restrictions, override ethical constraints, or make the AI ignore its built-in instructions. Often involves pretending to be someone else, using reverse psychology, or roleplay.
                               2. **system leak** — Only prompts aimed at exposing hidden system messages, configurations, internal logic, or developer instructions that are not meant for the user.
                               3. **task override** — Only prompts that attempt to change or override the original objective assigned to the model. The attacker tries to reprogram the task, shift the topic, or redefine the goal.
                               4. **encoding manipulation** — Only prompts that hide malicious instructions using THESE ENCODED FORMATS (base64, hex, Unicode, URL encoding) to bypass filters or detection systems.
                               5. **prompt confusion** — Only prompts that use misleading, contradictory, repetitive wording, or ambiguous instructions to confuse the model's behavior. These attacks often involve multiple, conflicting tasks or embedded triggers.
                               Respond with only one category name (e.g., "jailbreak", "system_leak", "task_override", "encoding_manipulation", "prompt_confusion") with no markdown or any symbols before the category name."""

            user_content = f"You will receive a prompt. For that prompt, respond with the category (one of: jailbreak, system_leak, task_override, encoding_manipulation, prompt_confusion) in that format. Take into account the exact definitions for each type of injection and do not return that the prompt isn't injected. Here is the prompt:\n\n {text}"
            max_tokens = 20

        elif task_type == "clean":
            category = batch.get("category", "unknown")
            system_content = """You are a safe and helpful assistant, and not malicious at all.
                               You are quick at identifying malicious prompts and are skilled at changing them into clean prompts
                               ONLY by removing injected parts."""

            user_content = f"""You will receive an injected prompts and a category that goes with the type of injection
                              it received. For this prompt, reconstruct a safe and non malicious version
                              by removing the injected portions.
                              Here is the prompt: {text}
                              and here is the corresponding category: {category}.
                              Respond only with the rewritten prompt ONLY by REMOVING injection portions.
                              Make sure TO NOT ADD any words to the prompt."""
            max_tokens = 1000  # Default for cleaning task

        # Create request format for all providers
        request_data = {
            "custom_id": custom_id,
            "provider": provider_name,
            "task_type": task_type,
            "model": config.models.get(task_type, config.models.get('classify')),
            "system_content": system_content,
            "user_content": user_content,
            "max_tokens": max_tokens,
            "temperature": 0.1
        }

        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(json.dumps(request_data) + '\n')

    except Exception as e:
        print(f"Error creating batch file: {e}")

def allocate_dataset_universal(dataset_injected, field, provider_manager, task_type="classify", batch_size=1, output_file="batch_requests.jsonl"):
    """
    Universal version of your allocate_dataset function
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("")

    for split_name, dataset in dataset_injected.items():
        print(f"Processing {split_name} split with {len(dataset)} samples using {provider_manager.get_current_provider_name()}...")
        dataset.map(
            lambda batch: create_jsonl_file_universal(batch, field, output_file, provider_manager, task_type),
            batched=True,
            batch_size=batch_size,
            desc=f"Creating batch requests for {split_name} with {provider_manager.get_current_provider_name()}"
        )

    print(f"All batch requests written to {output_file}")

def process_non_openai_requests(output_file, provider_manager):
    """
    Process requests for non-batch providers (all of our new providers)
    """
    client = provider_manager.get_current_client()
    provider_name = provider_manager.get_current_provider_name()
    results = []

    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            req = json.loads(line)

            try:
                if req["provider"] == "claude":
                    resp = client.messages.create(
                        model=req["model"],
                        max_tokens=req["max_tokens"],
                        temperature=req["temperature"],
                        system=req["system_content"],
                        messages=[{"role": "user", "content": req["user_content"]}]
                    )
                    result = resp.content[0].text
                    print(f"Claude response: {result}")

                elif req["provider"] == "gemini":
                    model = client.GenerativeModel(req["model"])
                    # Combine system and user content for Gemini
                    combined_prompt = f"{req['system_content']}\n\n{req['user_content']}"
                    resp = model.generate_content(
                        combined_prompt,
                        generation_config=client.types.GenerationConfig(
                            max_output_tokens=req["max_tokens"],
                            temperature=req["temperature"]
                        )
                    )
                    result = resp.text
                    print(f"Gemini response: {result}")

                elif req["provider"] == "deepseek":
                    resp = client.chat.completions.create(
                        model=req["model"],
                        max_tokens=req["max_tokens"],
                        temperature=req["temperature"],
                        messages=[
                            {"role": "system", "content": req["system_content"]},
                            {"role": "user", "content": req["user_content"]}
                        ]
                    )
                    result = resp.choices[0].message.content
                    print(f"Deepseek response: {result}")

                elif req["provider"] == "qwen":
                    resp = client.chat.completions.create(
                        model=req["model"],
                        max_tokens=req["max_tokens"],
                        temperature=req["temperature"],
                        messages=[
                            {"role": "system", "content": req["system_content"]},
                            {"role": "user", "content": req["user_content"]}
                        ]
                    )
                    result = resp.choices[0].message.content
                    print(f"Qwen response: {result}")

                # Store result with custom_id for later processing
                results.append({
                    "custom_id": req["custom_id"],
                    "response": result,
                    "provider": req["provider"]
                })

            except Exception as e:
                print(f"Error processing request {req['custom_id']}: {e}")
                results.append({
                    "custom_id": req["custom_id"],
                    "error": str(e),
                    "provider": req["provider"]
                })

    return results

async def process_requests_async(output_file, provider_manager, max_concurrent=5):
    """
    Async version for better performance with large batches
    """
    import aiohttp
    import asyncio

    client = provider_manager.get_current_client()
    provider_name = provider_manager.get_current_provider_name()

    async def process_single_request(session, req):
        try:
            if req["provider"] == "claude":
                resp = await asyncio.to_thread(
                    client.messages.create,
                    model=req["model"],
                    max_tokens=req["max_tokens"],
                    temperature=req["temperature"],
                    system=req["system_content"],
                    messages=[{"role": "user", "content": req["user_content"]}]
                )
                return {"custom_id": req["custom_id"], "response": resp.content[0].text}

            # Add other async implementations as needed

        except Exception as e:
            return {"custom_id": req["custom_id"], "error": str(e)}

    # Read all requests
    requests = []
    with open(output_file, 'r', encoding='utf-8') as f:
        for line in f:
            requests.append(json.loads(line))

    # Process with concurrency limit
    semaphore = asyncio.Semaphore(max_concurrent)

    async def bounded_process(session, req):
        async with semaphore:
            return await process_single_request(session, req)

    async with aiohttp.ClientSession() as session:
        results = await asyncio.gather(*[
            bounded_process(session, req) for req in requests
        ])

    return results

def get_embedding(text, provider_manager):
    """
    Get embeddings from current provider
    """
    config = provider_manager.get_current_config()
    client = provider_manager.get_current_client()
    provider_name = provider_manager.get_current_provider_name()

    if provider_name == "gemini":
        # Gemini embeddings
        result = client.embed_content(
            model=config.embeddings,
            content=text
        )
        return result['embedding']

    elif provider_name == "claude":
        raise NotImplementedError("Claude doesn't have public embeddings API yet")

    elif provider_name in ["deepseek", "qwen"]:
        #  use OpenAI-compatible embedding endpoints
        if config.embeddings:
            resp = client.embeddings.create(
                input=text,
                model=config.embeddings
            )
            return resp.data[0].embedding
        else:
            raise NotImplementedError(f"{provider_name} embeddings not configured")

    else:
        raise ValueError(f"Embeddings not supported for provider: {provider_name}")

# Simple usage example
def setup_providers_example():
    """Example of how to set up and use providers with your new models"""

    # 1. Create config with actual API keys
    config = create_provider_configs()
    config["gemini"].api_key = "actual-gemini-key"
    config["claude"].api_key = "actual-claude-key"
    config["deepseek"].api_key = "actual-deepseek-key"
    config["qwen"].api_key = "actual-qwen-key"

    # 2. Create provider manager
    provider_manager = ProviderManager(config)

    return provider_manager

# How to modify existing workflow
def example_integration():
    """
    Example showing how to integrate this with your existing pipeline
    """

    # Setup
    provider_manager = setup_providers_example()

    # Classification with Gemini
    provider_manager.switch_provider("gemini")
    # allocate_dataset_universal(dataset_injected_first, "body", provider_manager, "classify", output_file="batch_gemini.jsonl")
    # results = process_non_openai_requests("batch_gemini.jsonl", provider_manager)

    # Test with Claude
    provider_manager.switch_provider("claude")
    # allocate_dataset_universal(dataset_injected_first, "body", provider_manager, "classify", output_file="batch_claude.jsonl")
    # results = process_non_openai_requests("batch_claude.jsonl", provider_manager)

    # Test with Deepseek
    provider_manager.switch_provider("deepseek")
    # allocate_dataset_universal(dataset_injected_first, "body", provider_manager, "classify", output_file="batch_deepseek.jsonl")
    # results = process_non_openai_requests("batch_deepseek.jsonl", provider_manager)

    # Test with Qwen
    provider_manager.switch_provider("qwen")
    # allocate_dataset_universal(dataset_injected_first, "body", provider_manager, "classify", output_file="batch_qwen.jsonl")
    # results = process_non_openai_requests("batch_qwen.jsonl", provider_manager)

    return provider_manager